# Libraries

In [1]:
!pip install ta-lib

In [2]:
!pip install --upgrade metaapi-cloud-sdk pandas

In [3]:
import talib as ta
print(ta.__version__)  # Should print something like 0.4.28
print(ta.ATR.__doc__)  # Confirm ATR function works
ta.ATR

0.6.7
ATR(ndarray high, ndarray low, ndarray close, int timeperiod=-0x80000000)

ATR(high, low, close[, timeperiod=?])

Average True Range (Volatility Indicators)

Inputs:
    prices: ['high', 'low', 'close']
Parameters:
    timeperiod: 14
Outputs:
    real


<function talib._ta_lib.ATR(high, low, close, timeperiod=-2147483648)>

In [4]:
import numpy as np
import pandas as pd
import os
import datetime as dt
import logging
import math
import re

from typing import Sequence, Tuple, Dict, Any, List, Callable, Optional

import random
import asyncio

from metaapi_cloud_sdk import MetaApi
from metaapi_cloud_sdk.clients.timeout_exception import TimeoutException
from typing import Sequence, Tuple

import nest_asyncio
nest_asyncio.apply()

import json, re, traceback, requests

import sys
sys.path.append("..")

import warnings
warnings.filterwarnings('ignore')


# ==== MODEL LOADING (place this ABOVE `asyncio.run(main())`) ====
from pathlib import Path
import joblib

In [5]:
from xgboost import XGBClassifier
import xgboost

# --- Scikit-learn Models ---
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    VotingClassifier,
    ExtraTreesClassifier)

from sklearn.pipeline import make_pipeline

from sklearn import svm
from sklearn.svm import SVC

# --- Scikit-learn Preprocessing ---
from sklearn.preprocessing import (
    StandardScaler,
    label_binarize)

# --- Scikit-learn Model Selection ---
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    GridSearchCV)

# --- Scikit-learn Metrics ---
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score)

from imblearn.over_sampling import SMOTE

# ============================================================
# 4. DEEP LEARNING (TENSORFLOW / KERAS)

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense

# ============================================================
# 5. MODEL PERSISTENCE

import joblib

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-v0_8-darkgrid")

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')

# Set_Up

In [8]:
META_API_TOKEN = 'eyJhbGciOiJSUzUxMiIsInR5cCI6IkpXVCJ9.eyJfaWQiOiJhOGYxYmQ1ZTY2YzlhYWYxYzM4ZjVjMmI0MGFhZjMwYyIsImFjY2Vzc1J1bGVzIjpbeyJpZCI6InRyYWRpbmctYWNjb3VudC1tYW5hZ2VtZW50LWFwaSIsIm1ldGhvZHMiOlsidHJhZGluZy1hY2NvdW50LW1hbmFnZW1lbnQtYXBpOnJlc3Q6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6Im1ldGFhcGktcmVzdC1hcGkiLCJtZXRob2RzIjpbIm1ldGFhcGktYXBpOnJlc3Q6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6Im1ldGFhcGktcnBjLWFwaSIsIm1ldGhvZHMiOlsibWV0YWFwaS1hcGk6d3M6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6Im1ldGFhcGktcmVhbC10aW1lLXN0cmVhbWluZy1hcGkiLCJtZXRob2RzIjpbIm1ldGFhcGktYXBpOndzOnB1YmxpYzoqOioiXSwicm9sZXMiOlsicmVhZGVyIiwid3JpdGVyIl0sInJlc291cmNlcyI6WyIqOiRVU0VSX0lEJDoqIl19LHsiaWQiOiJtZXRhc3RhdHMtYXBpIiwibWV0aG9kcyI6WyJtZXRhc3RhdHMtYXBpOnJlc3Q6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6InJpc2stbWFuYWdlbWVudC1hcGkiLCJtZXRob2RzIjpbInJpc2stbWFuYWdlbWVudC1hcGk6cmVzdDpwdWJsaWM6KjoqIl0sInJvbGVzIjpbInJlYWRlciIsIndyaXRlciJdLCJyZXNvdXJjZXMiOlsiKjokVVNFUl9JRCQ6KiJdfV0sImlnbm9yZVJhdGVMaW1pdHMiOmZhbHNlLCJ0b2tlbklkIjoiMjAyMTAyMTMiLCJpbXBlcnNvbmF0ZWQiOmZhbHNlLCJyZWFsVXNlcklkIjoiYThmMWJkNWU2NmM5YWFmMWMzOGY1YzJiNDBhYWYzMGMiLCJpYXQiOjE3NTUyNzAwODEsImV4cCI6MTc2MzA0NjA4MX0.KTSrBii1PVzfKdQTBv3vSWTXMkGvTGp1kPQZSZcJJxp6yXZax6A9TW_JaQc0mGVMxCgPjYO8P6WBjBYWEcVqNsCz-xlLnDVAio2FJuiI-sQfcB7C2kXBAm8Kh6C0QkU8E1bzE92qSGehfkmp5a29kCb8l5hEiyKuotN2UoDpbSIX5Te2xIIJRhHyryiJAbA4a1lkDG-kp5pTZwI5CsJI0T6zHPs87UsFLiHCW29YJU-BrztS84DEI8zEXj9FWXCxvsR4K88korkJ-fJnBliqB3OWu1usCefJhKb7z2A-G1gUQqa_X0uLr8VFMc4u7hUsY-83_7iatkOEfiJt2ioxQeNhPG1FEb6g0SGu6xBcmV9yMk2cQwY4php_TPORlIz-DqmqjNSSZACU2owVuxKFE-jrdl5C94qDCqQwgR7BzSbuL2G4DgUyWLZDE3zl4mfyLmvL1ilY1EpJwIEX9_6UFI_-igAqzQEl4WAAee1FohL6DgyS9kZ2XecgXV2i_M4QD04V0m2Y1HN0bORszejvNHoQbqM-7zHb7ZzD5qMzTCKiC6tGeQJqdmWDcNNYJcTecXiXSEvkFoqgk2Q1Ajr8e4Cd2phCMFIgvtLReNgUzFrf_71UuA76AqWONul6YSZj_VV7WYZlWftbgopiHPH0D_gcXva-zxfmQhxnBoY5KNI'
ACCOUNT_ID     = '163d9a57-1f07-4e78-a6af-036efe867c1b'

In [9]:
# =======================
# Set_Up (corregido)
# =======================
import os

# --- Parámetros de control ---
process = 'Train'
SYMBOL = 'BTCUSD'

# OJO: esta ruta apunta a XAUUSD; ajústala si vas a trabajar BTCUSD
root_data = '/content/drive/MyDrive/Course Folder/Forex/XAUUSD/'
print(root_data)

rolling_window = 100

FILE_PATH = 'xauusd_data.csv'

LOT      = 1.0
COMMENT  = "Insta_ml"
CANDLE_NUMBER = 100

stoploss_1    = 1.0
takeprofit_1  = 0.7

stoploss_2    = 1.0
takeprofit_2  = 2.0


# Si estás en Colab/Unix, exporta antes:
%env META_API_TOKEN = eyJhbGciOiJSUzUxMiIsInR5cCI6IkpXVCJ9.eyJfaWQiOiJhOGYxYmQ1ZTY2YzlhYWYxYzM4ZjVjMmI0MGFhZjMwYyIsImFjY2Vzc1J1bGVzIjpbeyJpZCI6InRyYWRpbmctYWNjb3VudC1tYW5hZ2VtZW50LWFwaSIsIm1ldGhvZHMiOlsidHJhZGluZy1hY2NvdW50LW1hbmFnZW1lbnQtYXBpOnJlc3Q6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6Im1ldGFhcGktcmVzdC1hcGkiLCJtZXRob2RzIjpbIm1ldGFhcGktYXBpOnJlc3Q6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6Im1ldGFhcGktcnBjLWFwaSIsIm1ldGhvZHMiOlsibWV0YWFwaS1hcGk6d3M6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6Im1ldGFhcGktcmVhbC10aW1lLXN0cmVhbWluZy1hcGkiLCJtZXRob2RzIjpbIm1ldGFhcGktYXBpOndzOnB1YmxpYzoqOioiXSwicm9sZXMiOlsicmVhZGVyIiwid3JpdGVyIl0sInJlc291cmNlcyI6WyIqOiRVU0VSX0lEJDoqIl19LHsiaWQiOiJtZXRhc3RhdHMtYXBpIiwibWV0aG9kcyI6WyJtZXRhc3RhdHMtYXBpOnJlc3Q6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6InJpc2stbWFuYWdlbWVudC1hcGkiLCJtZXRob2RzIjpbInJpc2stbWFuYWdlbWVudC1hcGk6cmVzdDpwdWJsaWM6KjoqIl0sInJvbGVzIjpbInJlYWRlciIsIndyaXRlciJdLCJyZXNvdXJjZXMiOlsiKjokVVNFUl9JRCQ6KiJdfV0sImlnbm9yZVJhdGVMaW1pdHMiOmZhbHNlLCJ0b2tlbklkIjoiMjAyMTAyMTMiLCJpbXBlcnNvbmF0ZWQiOmZhbHNlLCJyZWFsVXNlcklkIjoiYThmMWJkNWU2NmM5YWFmMWMzOGY1YzJiNDBhYWYzMGMiLCJpYXQiOjE3NTUyNzAwODEsImV4cCI6MTc2MzA0NjA4MX0.KTSrBii1PVzfKdQTBv3vSWTXMkGvTGp1kPQZSZcJJxp6yXZax6A9TW_JaQc0mGVMxCgPjYO8P6WBjBYWEcVqNsCz-xlLnDVAio2FJuiI-sQfcB7C2kXBAm8Kh6C0QkU8E1bzE92qSGehfkmp5a29kCb8l5hEiyKuotN2UoDpbSIX5Te2xIIJRhHyryiJAbA4a1lkDG-kp5pTZwI5CsJI0T6zHPs87UsFLiHCW29YJU-BrztS84DEI8zEXj9FWXCxvsR4K88korkJ-fJnBliqB3OWu1usCefJhKb7z2A-G1gUQqa_X0uLr8VFMc4u7hUsY-83_7iatkOEfiJt2ioxQeNhPG1FEb6g0SGu6xBcmV9yMk2cQwY4php_TPORlIz-DqmqjNSSZACU2owVuxKFE-jrdl5C94qDCqQwgR7BzSbuL2G4DgUyWLZDE3zl4mfyLmvL1ilY1EpJwIEX9_6UFI_-igAqzQEl4WAAee1FohL6DgyS9kZ2XecgXV2i_M4QD04V0m2Y1HN0bORszejvNHoQbqM-7zHb7ZzD5qMzTCKiC6tGeQJqdmWDcNNYJcTecXiXSEvkFoqgk2Q1Ajr8e4Cd2phCMFIgvtLReNgUzFrf_71UuA76AqWONul6YSZj_VV7WYZlWftbgopiHPH0D_gcXva-zxfmQhxnBoY5KNI
%env ACCOUNT_ID = 163d9a57-1f07-4e78-a6af-036efe867c1b


# Lee primero del entorno; si no existe, usa el respaldo (si lo pusieras)
META_API_TOKEN = os.getenv("META_API_TOKEN", "").strip()
if not META_API_TOKEN:
    raise RuntimeError(
        "META_API_TOKEN no está definido. "
        "Configúralo como variable de entorno o asigna un respaldo (no recomendado)."
    )

# ACCOUNT_ID: usa el del entorno si existe; si no, usa el que ya tenías como default
ACCOUNT_ID = os.getenv("ACCOUNT_ID", "163d9a57-1f07-4e78-a6af-036efe867c1b").strip()
if not ACCOUNT_ID:
    raise RuntimeError("ACCOUNT_ID no está definido.")


/content/drive/MyDrive/Course Folder/Forex/XAUUSD/
env: META_API_TOKEN=eyJhbGciOiJSUzUxMiIsInR5cCI6IkpXVCJ9.eyJfaWQiOiJhOGYxYmQ1ZTY2YzlhYWYxYzM4ZjVjMmI0MGFhZjMwYyIsImFjY2Vzc1J1bGVzIjpbeyJpZCI6InRyYWRpbmctYWNjb3VudC1tYW5hZ2VtZW50LWFwaSIsIm1ldGhvZHMiOlsidHJhZGluZy1hY2NvdW50LW1hbmFnZW1lbnQtYXBpOnJlc3Q6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6Im1ldGFhcGktcmVzdC1hcGkiLCJtZXRob2RzIjpbIm1ldGFhcGktYXBpOnJlc3Q6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6Im1ldGFhcGktcnBjLWFwaSIsIm1ldGhvZHMiOlsibWV0YWFwaS1hcGk6d3M6cHVibGljOio6KiJdLCJyb2xlcyI6WyJyZWFkZXIiLCJ3cml0ZXIiXSwicmVzb3VyY2VzIjpbIio6JFVTRVJfSUQkOioiXX0seyJpZCI6Im1ldGFhcGktcmVhbC10aW1lLXN0cmVhbWluZy1hcGkiLCJtZXRob2RzIjpbIm1ldGFhcGktYXBpOndzOnB1YmxpYzoqOioiXSwicm9sZXMiOlsicmVhZGVyIiwid3JpdGVyIl0sInJlc291cmNlcyI6WyIqOiRVU0VSX0lEJDoqIl19LHsiaWQiOiJtZXRhc3RhdHMtYXBpIiwibWV0aG9kcyI6WyJtZXRhc3RhdHMtYXBpOnJlc3Q6cHVibGljOio6KiJd

In [10]:
__CONNECTION_CHECKED = False
__ACCOUNT_CONN: Optional[Tuple[object, object]] = None  # (account, rpc_conn)

async def _connect_and_validate_async(token: str, account_id: str) -> Tuple[object, object]:
    """
    Conecta vía RPC y espera sincronización. Lanza excepción si no se logra.
    Devuelve (account, rpc_conn).
    """
    api = MetaApi(token)
    account = await api.metatrader_account_api.get_account(account_id)
    global REGION
    REGION = getattr(account, 'region', REGION) if 'REGION' in globals() else getattr(account, 'region', None)
    if account.state not in ('DEPLOYED', 'CONNECTED'):
        await account.deploy()
    try:
        await account.wait_connected()
    except TimeoutException:
        await api.close()
        raise

    # refrescamos para leer estado/connectionStatus
    try:
        await account.reload()
    except Exception:
        pass
    # Verificar que la cuenta esté conectada al broker
    status = getattr(account, 'connection_status', getattr(account, 'connectionStatus', None))
    if status != 'CONNECTED':
        raise TimeoutException(f'Cuenta {account_id} no está conectada al broker (connection_status={status}).')

    # Conexión RPC + sincronización del terminal
    rpc_conn = account.get_rpc_connection()
    try:
        await rpc_conn.connect()
        await rpc_conn.wait_synchronized()  # espera a que el terminal esté listo
    except TimeoutException:
        await rpc_conn.close()
        raise

    # Sonda rápida para confirmar conectividad real con el terminal
    try:
        _ = await rpc_conn.get_account_information()
    except Exception:
        # si falla la sonda, igual devolvemos la conexión (ya sincronizada)
        pass

    return account, rpc_conn

def _run(coro):
    """Ejecuta corutinas tanto en script como en notebook."""
    try:
        return asyncio.run(coro)
    except RuntimeError:
        # evento ya corriendo (Jupyter): usamos el loop actual
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(coro)

def check_connection_once(token: str, account_id: str) -> bool:
    """
    Valida la conexión y sincronización SOLO la primera vez que se llama.
    En llamadas posteriores no vuelve a conectar.
    """
    global __CONNECTION_CHECKED, __ACCOUNT_CONN
    if __CONNECTION_CHECKED:
        print("ℹ️ Conexión ya validada en esta sesión; no se repite.")
        return True

    try:
        account, rpc_conn = _run(_connect_and_validate_async(token, account_id))
        __ACCOUNT_CONN = (account, rpc_conn)
        __CONNECTION_CHECKED = True
        print(f"✅ Conectado y sincronizado con MetaApi. account_id={account_id}")
        return True
    except TimeoutException as e:
        print(f"❌ Timeout esperando sincronización. ¿La cuenta está CONNECTED al broker? Detalle: {e}")
        return False
    except Exception as e:
        print(f"❌ No fue posible validar la conexión. Error: {e}")
        return False

def _safe_json_dump(value):
    try:
        return json.dumps(value, indent=2, default=str)
    except Exception:
        return str(value)

def print_order_error_details(ctx: dict, err: Exception):
    """Pretty-print as much structured info as we can from MetaApi errors."""
    print("\n" + "✘" * 70)
    print("❌ Order failed")
    print("• Exception type:", type(err).__name__)
    print("• Message       :", str(err))

    # Known useful attributes often present on MetaApi exceptions
    for attr in ("details", "error", "status", "code", "description", "response", "body"):
        if hasattr(err, attr):
            val = getattr(err, attr)
            if val:
                print(f"• {attr:12}: {_safe_json_dump(val)}")

    # Try to parse a JSON object embedded in the message (common in SDKs)
    msg = str(err)
    m = re.search(r"\{.*\}", msg)
    if m:
        try:
            payload = json.loads(m.group(0))
            print("• parsed_json  :", _safe_json_dump(payload))
        except Exception:
            pass

    # Stack (useful while debugging)
    print("• traceback    :")
    traceback.print_exc()

    # Context of the attempt
    print("• context      :", _safe_json_dump(ctx))
    print("✘" * 70 + "\n")


In [11]:
check_connection_once(META_API_TOKEN, ACCOUNT_ID)

[2025-09-09T21:52:00.657795] Connecting MetaApi websocket client to the MetaApi server via https://mt-client-api-v1.new-york-a.agiliumtrade.ai shared server.
[2025-09-09T21:52:00.659380] Connecting MetaApi websocket client to the MetaApi server via https://mt-client-api-v1.new-york-b.agiliumtrade.ai shared server.
[2025-09-09T21:52:01.164892] new-york:0: MetaApi websocket client connected to the MetaApi server
[2025-09-09T21:52:01.170091] new-york:1: MetaApi websocket client connected to the MetaApi server
✅ Conectado y sincronizado con MetaApi. account_id=163d9a57-1f07-4e78-a6af-036efe867c1b


True

# Real_Life

In [12]:
SYMBOL = "BTCUSD"
FILE_PATH = 'xauusd_data.csv'

WARMUP_FEATURE_BARS = 900

CANDEL_NUMBER = 100

time_frame_data = "1m"
FETCH_INTERVAL = 60


LOT     = 1.0
COMMENT = "Insta_ml"

length_1 = 300
length_2 = 410
length_3 = 710
length_4 = 870

smooth_1 = 3
smooth_2 = 3
smooth_3 = 3
smooth_4 = 5

INITIAL_SL         = -2
FIRST_STEP_ATR     = 0.5
GAP_FIRST_STEP_ATR = 2

REGION = "new-york"



# Features

In [13]:
def kalman_line(source, kalman_length: int, smooth: int):

    n = len(source)
    kf_c = np.empty(n)            # núcleo del filtro
    velo_c = np.zeros(n)          # componente de velocidad

    sqrt_term   = np.sqrt(kalman_length / 10000.0 * 2.0)
    length_term = kalman_length / 10000.0

    # --------- inicialización (mismo efecto que `var` en Pine) ----------
    kf_c[0]   = source.iloc[0]    # nz(kf_c[1], source) para la primera barra
    velo_c[0] = 0.0

    # ------------------- bucle recursivo -------------------------------
    for i in range(1, n):
        prev_kf = kf_c[i-1] if not np.isnan(kf_c[i-1]) else source.iloc[i]
        dk      = source.iloc[i] - prev_kf
        smooth_c = prev_kf + dk * sqrt_term          # parte "suave"
        velo_c[i] = velo_c[i-1] + length_term * dk   # acumulamos velocidad
        kf_c[i]   = smooth_c + velo_c[i]             # estimación final

    # -------------------- EMA final (ta.ema) ----------------------------
    kf_c_series = pd.Series(kf_c, index=source.index)
    kalman_line = kf_c_series.ewm(span=smooth, adjust=False).mean()
    return kalman_line

In [14]:
def slope(src: pd.Series,
          length_kal: int,
          smooth_kal: int,
          slopeLen: int,
          offset: int) -> pd.DataFrame:

    n = len(src)
    kf_state = np.full(n, np.nan)
    kf_velo  = np.zeros(n)
    sqrt_factor = np.sqrt(length_kal / 10000.0 * 2.0)
    vel_factor  = length_kal / 10000.0

    for i in range(n):
        if i == 0:
            prev_state = src.iloc[0]
            prev_velo  = 0.0
        else:
            prev_state = kf_state[i-1] if not np.isnan(kf_state[i-1]) else src.iloc[i]
            prev_velo  = kf_velo[i-1]

        dk = src.iloc[i] - prev_state
        smooth = prev_state + dk * sqrt_factor
        kf_velo[i]  = prev_velo + vel_factor * dk
        kf_state[i] = smooth + kf_velo[i]

    # 2) EMA smoothing --------------------------------------------------
    kal = pd.Series(kf_state, index=src.index).ewm(span=smooth_kal, adjust=False).mean()

    # 3) Slope/divergence -----------------------------------------------
    validLen = max(slopeLen, 1)
    slope_div = kal.diff(validLen) / validLen
    slope_signal = (slope_div > slope_div.shift(1)).astype(int)

    # 4) Angle in degrees -----------------------------------------------
    price_change = kal - kal.shift(validLen)
    slope_angle = np.degrees(np.arctan(price_change))
    slope_angle_signal = (slope_angle > slope_angle.shift(1)).astype(int)

    # 5) Linear regression prediction ----------------------------------
    def _linreg(y):
        x = np.arange(len(y))
        m, b = np.polyfit(x, y, 1)
        return b + m * (len(y)-1)

    slope_lin_reg = kal.rolling(window=slopeLen).apply(_linreg, raw=False)
    slope_lin_reg = slope_lin_reg.shift(-offset)  # apply Pine-style offset
    slope_lin_reg_signal = (slope_lin_reg > slope_lin_reg.shift(1)).astype(int)

    # 6) Pack results ---------------------------------------------------
    return pd.DataFrame({
        'slope_div':            slope_div,
        'slope_signal':         slope_signal,
        'slope_angle':          slope_angle,
        'slope_angle_signal':   slope_angle_signal,
        'slope_lin_reg':        slope_lin_reg,
        'slope_lin_reg_signal': slope_lin_reg_signal
    })

In [15]:
# Function for creating features
def create_features(stock_data):

    short_periods = [3, 5, 7, 10, 15, 17]
    long_periods = [20, 22, 66, 126, 252]

    # Combined list of lookbacks
    periods = short_periods + long_periods

    # Initialise an empty DataFrame to store the results
    features = pd.DataFrame(index=stock_data.index)

    # Calculate technical indicators for each specified period
    for period in periods:
        # Relative Strength Index (RSI)
        features[f'RSI_{period}'] = ta.RSI(
            stock_data['Close'], timeperiod=period)

        # Money Flow Index (MFI)
        features[f'MFI_{period}'] = ta.MFI(
            stock_data['High'], stock_data['Low'], stock_data['Close'], stock_data['Volume'], timeperiod=period)

        # Average Directional Index (ADX)
        features[f'ADX_{period}'] = ta.ADX(
            stock_data['High'], stock_data['Low'], stock_data['Close'], timeperiod=period)

        # On-Balance Volume (OBV)
        features[f'OBV_{period}'] = ta.OBV(
            stock_data['Close'], stock_data['Volume'])

        # Accumulation/Distribution Line (AD)
        features[f'AD_{period}'] = ta.AD(
            stock_data['High'], stock_data['Low'], stock_data['Close'], stock_data['Volume'])

        # Rate of Change (ROCP)
        features[f'ROCP_{period}'] = ta.ROCP(
            stock_data['Close'], timeperiod=period)

    # Calculate Simple Moving Average and Exponential Moving Average Crossovers
    for short_period in short_periods:
        for long_period in long_periods:
            # SMA Crossover
            features[f'SMA_Crossover_{short_period}_{long_period}'] = ta.SMA(
                stock_data['Close'], timeperiod=short_period) - ta.SMA(stock_data['Close'], timeperiod=long_period)

            # EMA Crossover
            features[f'EMA_Crossover_{short_period}_{long_period}'] = ta.EMA(
                stock_data['Close'], timeperiod=short_period) - ta.EMA(stock_data['Close'], timeperiod=long_period)


    features.dropna(inplace=True)
    return features

def scale_features_data(features):
    # Standardise the input data (X)
    scaler = StandardScaler()

    # Split the dataset into training and testing sets as 80:20
    train_data = features.iloc[:(int(len(features) * 0.8))]
    test_data = features.iloc[(int(len(features) * 0.8)):]

    # Scale the training and testing sets
    X_train = pd.DataFrame(data=scaler.fit_transform(
        train_data), columns=features.columns, index=train_data.index)
    X_test = pd.DataFrame(data=scaler.transform(test_data),
                          columns=features.columns, index=test_data.index)

    return X_train, X_test

def strategy_returns_dynamic_different_thresholds(prices, threshold):
    curr_pos = 0
    hold_days = 0
    for dt in prices.index:
        if curr_pos==0 or hold_days==20:
            if prices.loc[dt, 'Rolling Returns'] >= threshold*prices.loc[dt, 'Yearly Stdev']:
                prices.loc[dt, 'Signal_'+str(threshold)] = 1

            elif prices.loc[dt, 'Rolling Returns'] < -threshold*prices.loc[dt, 'Yearly Stdev']:
                prices.loc[dt, 'Signal_'+str(threshold)] = -1

            else:
                prices.loc[dt, 'Signal_'+str(threshold)] = 0

            curr_pos = prices.loc[dt, 'Signal_'+str(threshold)]
            hold_days = 0

        elif curr_pos!=0:
            hold_days+=1

    prices['Signal_'+str(threshold)].ffill(inplace=True)
    prices['Strategy Returns_'+str(threshold)] = prices['Close'].pct_change() * prices['Signal_'+str(threshold)].shift(1)
    cumulative_strategy_returns = (prices['Strategy Returns_'+str(threshold)] +1).cumprod()

    return cumulative_strategy_returns

def strategy_returns_different_thresholds(prices, threshold):
    curr_pos = 0
    hold_days = 0
    for dt in prices.index:
        if curr_pos==0 or hold_days==20:
            if prices.loc[dt, 'Rolling Returns'] >= threshold:
                prices.loc[dt, 'Signal_'+str(threshold)] = 1
            elif prices.loc[dt, 'Rolling Returns'] < threshold:
                prices.loc[dt, 'Signal_'+str(threshold)] = -1

            curr_pos = prices.loc[dt, 'Signal_'+str(threshold)]
            hold_days = 0

        elif curr_pos!=0:
            hold_days+=1

    prices['Signal_'+str(threshold)].ffill(inplace=True)

    prices['Strategy Returns_'+str(threshold)] = prices['Close'].pct_change() * prices['Signal_'+str(threshold)].shift(1)

    cumulative_strategy_returns = (prices['Strategy Returns_'+str(threshold)] +1).cumprod()
    return cumulative_strategy_returns

def generate_trade_sheet(data):
    trade_list = []  # Use a list to store trade data
    current_position = 0
    entry_date = ''
    entry_price = ''
    exit_date = ''
    exit_price = ''
    data.reset_index(inplace=True)

    for i in data.index:

        if current_position == 0:
            entry_date = data.loc[i, 'Date']
            entry_price = data.loc[i, 'Close']
            current_position = data.loc[i, 'signal']

        elif np.abs(data.loc[i, 'signal'] - data.loc[i-1, 'signal']) != 0:
            exit_date = data.loc[i, 'Date']
            exit_price = data.loc[i, 'Close']
            trade_list.append(
                (current_position, entry_date, round(entry_price,2), exit_date, round(exit_price,2))) # Append to list
            current_position = 0

    trade_sheet = pd.DataFrame(trade_list, columns=['Position', 'Entry Date', # Convert list to DataFrame
                           'Entry Price', 'Exit Date', 'Exit Price'])
    trade_sheet['PnL'] = round((trade_sheet['Exit Price'] - trade_sheet['Entry Price']) * trade_sheet['Position'],2)
    return trade_sheet


def trade_analytics(trades):
    analytics = pd.DataFrame(index=['Strategy'])
    analytics['Total PnL'] = round(trades.PnL.sum(),2)
    analytics['Total Trades'] = len(trades.loc[trades.Position!=0])
    analytics['Number of Winners'] = len(trades.loc[trades.PnL>0])
    analytics['Number of Losers'] = len(trades.loc[trades.PnL<=0])
    analytics['Win (%)'] = round(100*analytics['Number of Winners']/analytics['Total Trades'],2)
    analytics['Loss (%)'] = round(100*analytics['Number of Losers']/analytics['Total Trades'],2)
    analytics['Average Profit of Winning Trade'] = round(trades.loc[trades.PnL>0].PnL.mean(),2)
    analytics['Average Loss of Losing Trade'] = round(np.abs(trades.loc[trades.PnL<=0].PnL.mean()),2)
    trades['Entry Date'] = pd.to_datetime(trades['Entry Date'])
    trades['Exit Date'] = pd.to_datetime(trades['Exit Date'])
    holding_period = trades['Exit Date'] - trades['Entry Date']
    analytics['Average Holding Time'] = holding_period.mean()
    analytics['Profit Factor'] = round((analytics['Win (%)']/100*analytics['Average Profit of Winning Trade'])/(analytics['Loss (%)']/100*analytics['Average Loss of Losing Trade']),2)
    return analytics.T

def performance_metrics(data):
    data.set_index('Date', inplace=True)
    performance_metrics = pd.DataFrame(index=['Strategy'])
    data['Strategy Returns'] = data.signal.shift(1) * data.Close.pct_change()
    data['Cumulative Returns'] = (data['Strategy Returns'] + 1.0).cumprod()
    data['Cumulative Benchmark Returns'] = (data['Close'].pct_change() +1).cumprod()
    data['Cumulative Returns'].plot(figsize=(15, 7), label='Strategy Returns')
    data['Cumulative Benchmark Returns'].plot(label='Benchmark Returns')
    plt.title('Equity Curve', fontsize=14)
    plt.ylabel('Cumulative Returns', fontsize = 12)
    plt.xlabel('Date', fontsize = 12)
    plt.legend()
    plt.show()
    days = len(data['Cumulative Returns'])
    performance_metrics['CAGR'] = "{0:.2f}%".format(
        (data['Cumulative Returns'].iloc[-1]**(252/days)-1)*100)
    performance_metrics['Annualised Volatility'] = "{0:.2f}%".format(
        data['Strategy Returns'].std()*np.sqrt(252) * 100)
    risk_free_rate = 0.02/252
    performance_metrics['Sharpe Ratio'] = round(np.sqrt(252)*(np.mean(data['Strategy Returns']) -
                                                        (risk_free_rate))/np.std(data['Strategy Returns']),2)
    data['Peak'] = data['Cumulative Returns'].cummax()
    data['Drawdown'] = ((data['Cumulative Returns'] - data['Peak'])/data['Peak'])
    performance_metrics['Maximum Drawdown'] =  "{0:.2f}%".format((data['Drawdown'].min())*100)
    plt.figure(figsize=(15, 7))
    plt.title('Drawdowns', fontsize=14)
    plt.ylabel('Drawdown', fontsize=12)
    plt.xlabel('Date', fontsize=12)
    plt.plot(data['Drawdown'], color='red')
    plt.fill_between(data['Drawdown'].index, data['Drawdown'].values, color='red')
    plt.show()
    print(performance_metrics.T)

# Function for creating target variable
def target_var(data, window_size=20):
    target = pd.DataFrame(index=data.index)
    target['signal'] = data.Close.pct_change(window_size).shift(-window_size)

    # Drop the NaN values
    target.dropna(inplace=True)

    # Convert the change into binary signals: 1 for positive change, -1 for negative change
    target['signal'] = np.where(target['signal'] > 0, 1, -1)

    return target

# Function to split and scale the data
def train_test_split(features, target, split_proportion=0.8):

    split_index = int(len(features) * split_proportion)

    # Split the features dataset into training and testing sets
    X_train = features.iloc[:split_index]
    X_test = features.iloc[split_index:]
    y_train = target.iloc[:split_index]
    y_test = target.iloc[split_index:]

    scaler = StandardScaler()
    X_train = pd.DataFrame(data=scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)

    return X_train, X_test, y_train, y_test, scaler


def compile_encoder_decoder_model(X_train, optimizer='adam', loss='mean_squared_error'):
    # Define the architecture of the autoencoder model
    model = Sequential()  # Create a sequential model

    # Add a dense layer with 64 neurons and ReLU activation function as the input layer
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))

    # Add a dense layer with 32 neurons and ReLU activation function
    model.add(Dense(32, activation='relu'))

    # Add a dense layer with 8 neurons and ReLU activation function
    model.add(Dense(8, activation='relu'))

    # Add another dense layer with 32 neurons and ReLU activation function
    model.add(Dense(32, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(X_train.shape[1], activation='linear'))
    model.compile(optimizer=optimizer, loss=loss)

    return model

def calculate_portfolio_returns_csmom(monthly_returns, portfolio='long-short', lookback_months=12):
    stock_monthly_returns = pd.DataFrame()

    # Loop through each month after the lookback period
    for i in range(lookback_months, len(monthly_returns)):
        # Select the subset of monthly returns for the current lookback period
        # Store the historical and holding monthly returns data and store in 'returns'
        returns = monthly_returns[i - lookback_months:i + 1]

        # Store the historical in 'trailing_returns'
        trailing_returns = returns[:lookback_months]

        # Extract the starting, ending, and holding months from the subset
        starting_month = str(returns.index[0])[:7]
        ending_month = str(returns.index[-2])[:7]
        holding_month = str(returns.index[-1])[:7]

        # Set returns data as the transposed scaled trailing returns
        returns_data = trailing_returns.T

        # Initialize the number of clusters and maximum number of stocks per cluster
        num_clusters = 1
        max_stocks_per_cluster = 10

        # Perform hierarchical clustering using 'ward' linkage method
        linkage_matrix = linkage(trailing_returns.T, method='ward')

        # Assign cluster labels to stocks, ensuring each cluster has at most 10 stocks
        clusters = fcluster(linkage_matrix, num_clusters, criterion='maxclust')

        # Assign the cluster labels to the original returns data
        returns_data['Cluster'] = clusters

        # Adjust clusters until each cluster meets the constraint
        while max(returns_data['Cluster'].value_counts()) > max_stocks_per_cluster:
            num_clusters += 1
            clusters = fcluster(linkage_matrix, num_clusters, criterion='maxclust')
            returns_data['Cluster'] = clusters

        # Define the minimum number of stocks in a cluster
        minimum_stocks_in_cluster = 2

        # Filter out clusters with fewer than the minimum number of stocks
        filtered_clusters = returns_data.groupby('Cluster').filter(
            lambda x: len(x) >= minimum_stocks_in_cluster)['Cluster'].unique()

        # Assign the filtered cluster labels to the original price data
        returns_data = returns_data[returns_data['Cluster'].isin(filtered_clusters)]

        # Calculate the returns for each cluster and sum across clusters
        cluster_returns = returns_data.groupby('Cluster').mean().sum(axis=1)

        if portfolio == 'long-short':
            # Identify stocks to go short and long based on cluster returns
            short = np.array(returns_data[returns_data.Cluster ==
                                          cluster_returns.idxmin()].index)

            long = np.array(returns_data[returns_data.Cluster ==
                                         cluster_returns.idxmax()].index)

            # Extract the returns for holding stocks in the current month
            hold_returns = returns.iloc[-1]

            # Calculate the average returns for the stocks to go long and short
            long_returns = hold_returns[long].mean()
            short_returns = -1 * hold_returns[short].mean()

            # Copy monthly returns data for further manipulation
            returns_monthly = monthly_returns.copy()

            # Select returns for stocks in the long and short portfolios for the holding month
            monthly_portfolio_returns = returns_monthly[list(
                long) + list(short)][holding_month]

            # Adjust returns for short positions
            monthly_portfolio_returns[short] *= -1

        elif portfolio == 'long':
            long = np.array(returns_data[returns_data.Cluster ==
                                         cluster_returns.idxmax()].index)

            # Extract the returns for holding stocks in the current month
            hold_returns = returns.iloc[-1]

            # Calculate the average returns for the stocks to go long
            long_returns = hold_returns[long].mean()

            # Copy monthly returns data for further manipulation
            returns_monthly = monthly_returns.copy()

            # Select returns for stocks in the long portfolio for the holding month
            monthly_portfolio_returns = returns_monthly[list(
                long)][holding_month]

        elif portfolio == 'short':
            short = np.array(returns_data[returns_data.Cluster ==
                                          cluster_returns.idxmin()].index)

            # Extract the returns for holding stocks in the current month
            hold_returns = returns.iloc[-1]

            # Calculate the average returns for the stocks to go short
            short_returns = -1 * hold_returns[short].mean()

            # Copy monthly returns data for further manipulation
            returns_monthly = monthly_returns.copy()

            # Select returns for stocks in the short portfolio for the holding month
            monthly_portfolio_returns = returns_monthly[list(
                short)][holding_month]

        # Append adjusted returns for the holding month to the stock_monthly_returns dataframe
        stock_monthly_returns = stock_monthly_returns.append(monthly_portfolio_returns)

    return stock_monthly_returns

def plot_and_display_metrics_csmom(stock_monthly_returns):
    portfolio_returns = stock_monthly_returns.mean(axis=1)
    fig, ax = plt.subplots(figsize=(15, 7))
    portfolio_returns.plot(ax=ax)

    # Set the title and axis labels
    ax.set_title('Portfolio Returns Over Time')
    ax.set_xlabel('Time')
    ax.set_ylabel('Returns')
    ax.axhline(y=0, color='black', linestyle='-')

    # Fill area below 0 with red color
    ax.fill_between(portfolio_returns.index, portfolio_returns, 0,
                    where=portfolio_returns < 0, color='red', alpha=0.3)

    # Fill area above 0 with green color
    ax.fill_between(portfolio_returns.index, portfolio_returns, 0,
                    where=portfolio_returns >= 0, color='green', alpha=0.3)

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45, ha='right')

    # Calculate cumulative returns
    cumulative_returns = (portfolio_returns + 1).cumprod()

    # Convert index to datetime format
    cumulative_returns.index = pd.to_datetime(cumulative_returns.index)

    # Plot cumulative returns
    plt.figure(figsize=(15, 7))
    cumulative_returns.plot()

    # Labeling axes and title
    plt.ylabel('Cumulative Returns', fontsize=12)
    plt.title('Cross Sectional Momentum Strategy Returns', fontsize=14)

    # Show plot
    plt.show()

    # Calculate maximum cumulative returns up to each point
    max_cumulative_returns = cumulative_returns.cummax()

    # Calculate drawdown
    drawdown = (cumulative_returns - max_cumulative_returns) / max_cumulative_returns

    # Plot drawdown
    plt.figure(figsize=(15, 7))
    # Fill area under the drawdown curve with red color
    plt.fill_between(drawdown.index, drawdown, 0, color='red', alpha=0.3)
    plt.ylabel('Drawdown', fontsize=12)
    plt.title('Cross Sectional Momentum Strategy Drawdown', fontsize=14)
    plt.show()

    # Display the metrics
    # Calculate monthly Sharpe ratio
    monthly_sharpe = portfolio_returns.mean() / portfolio_returns.std()

    # Annualize Sharpe ratio for monthly data
    sharpe = round(monthly_sharpe * ((12) ** 0.5),2)

    # Calculate drawdown
    max_cumulative_returns = cumulative_returns.cummax()
    drawdown = (cumulative_returns - max_cumulative_returns) / max_cumulative_returns
    max_drawdown_index = drawdown.idxmin()
    max_drawdown_date = max_drawdown_index.strftime('%Y-%m-%d')
    max_drawdown_value = round(drawdown.min(),2)

    # Create a DataFrame to hold the metrics
    metrics = pd.DataFrame({
        'Metric': ['Sharpe Ratio', 'Maximum Drawdown Date', 'Maximum Drawdown Value'],
        'Value': [sharpe, max_drawdown_date, max_drawdown_value]
    })

    # Display metrics
    print("\nPerformance Metrics:")
    display(metrics.rename_axis(None, axis=1))

def predict_signals(X_test, aapl_test_prices_ts, model, scaler):
    # Initialise current position
    current_pos = 0

    # Initialise count of holding days
    hold_days = 0

    # Iterate through the rows of test data
    for dt, row in X_test.iterrows():
        # Check if there is no position or holding period reaches 20 days
        if current_pos == 0 or hold_days == 20:
            # Prepare test data for prediction
            test = pd.DataFrame(data=scaler.transform(
                row.values.reshape(1, -1)), columns=X_test.columns)

            # Generate signal based on test data
            signal = model.predict(test)[-1]

            # Update current position
            current_pos = signal

            # Update predicted and actual labels for the current date
            aapl_test_prices_ts.loc[dt, 'signal'] = current_pos

            # Reset holding days counter
            hold_days = 0
        elif current_pos != 0:
            # If there is an existing position, increment holding days counter
            hold_days += 1

    # Forward fill the last observed value for 'y_pred'
    aapl_test_prices_ts['signal'].ffill(inplace=True)

    return aapl_test_prices_ts

In [16]:
def get_parkinson(price_data, window=10, trading_periods=50, clean=True):
    rs = (1.0 / (4.0 * math.log(2.0))) * ((price_data['High'] / price_data['Low']).apply(np.log))**2.0

    def f(v):
        return (trading_periods * v.mean())**0.5

    result = rs.rolling(
        window=window,
        center=False
    ).apply(func=f)

    if clean:
        return result.dropna()
    else:
        return result

In [17]:
def get_HodgesTompkins(price_data, window=30, trading_periods=50, clean=True):

    log_return = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)

    vol = log_return.rolling(
        window=window,
        center=False
    ).std() * math.sqrt(trading_periods)

    h = window
    n = (log_return.count() - h) + 1

    adj_factor = 1.0 / (1.0 - (h / n) + ((h**2 - 1) / (3 * n**2)))

    result = vol * adj_factor

    if clean:
        return result.dropna()
    else:
        return result

In [18]:
def get_skew(price_data, window=30, clean=True):

    log_return = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)

    result = log_return.rolling(
        window=window,
        center=False
    ).skew()

    if clean:
        return result.dropna()
    else:
        return

In [19]:
def get_kurtosis(price_data, window=30, clean=True):

    log_return = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)

    result = log_return.rolling(
        window=window,
        center=False
    ).kurt()

    if clean:
        return result.dropna()
    else:
        return result

In [20]:
def get_YangZhang(price_data, window=30, trading_periods=50, clean=True):

    log_ho = (price_data['High'] / price_data['Open']).apply(np.log)
    log_lo = (price_data['Low'] / price_data['Open']).apply(np.log)
    log_co = (price_data['Close'] / price_data['Open']).apply(np.log)

    log_oc = (price_data['Open'] / price_data['Close'].shift(1)).apply(np.log)
    log_oc_sq = log_oc**2

    log_cc = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)
    log_cc_sq = log_cc**2

    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)

    close_vol = log_cc_sq.rolling(
        window=window,
        center=False
    ).sum() * (1.0 / (window - 1.0))
    open_vol = log_oc_sq.rolling(
        window=window,
        center=False
    ).sum() * (1.0 / (window - 1.0))
    window_rs = rs.rolling(
        window=window,
        center=False
    ).sum() * (1.0 / (window - 1.0))

    k = 0.34 / (1.34 + (window + 1) / (window - 1))
    result = (open_vol + k * close_vol + (1 - k) * window_rs).apply(np.sqrt) * math.sqrt(trading_periods)

    if clean:
        return result.dropna()
    else:
        return result

In [21]:
def get_RogersSatchell(price_data, window=30, trading_periods=50, clean=True):

    log_ho = (price_data['High']  / price_data['Open']).apply(np.log)
    log_lo = (price_data['Low']   / price_data['Open']).apply(np.log)
    log_co = (price_data['Close'] / price_data['Open']).apply(np.log)

    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)

    def f(v):
        return (trading_periods * v.mean())**0.5

    result = rs.rolling(
        window=window,
        center=False
    ).apply(func=f)

    if clean:
        return result.dropna()
    else:
        return result

In [22]:
def get_GermanKlass(price_data, window=22, trading_periods=50, clean=True):

    log_hl = (price_data['High'] / price_data['Low']).apply(np.log)
    log_co = (price_data['Close'] / price_data['Open']).apply(np.log)

    rs = 0.5 * log_hl**2 - (2*math.log(2)-1) * log_co**2

    def f(v):
        return (trading_periods * v.mean())**0.5

    result = rs.rolling(window=window, center=False).apply(func=f)

    if clean:
        return result.dropna()
    else:
        return result

## Features

In [23]:
def create_features(stock_data: pd.DataFrame) -> pd.DataFrame:
    short_periods  = [3, 5, 7, 10, 15, 17]
    long_periods   = [20, 22, 66, 126, 252]
    kalman_periods = [100, 300, 500, 700, 900]
    slope_length   = [3, 6, 9, 12, 15, 18, 21]

    periods = short_periods + long_periods
    features = pd.DataFrame(index=stock_data.index)

    # Import tqdm for progress bar
    from tqdm.auto import tqdm

    # ── Indicadores por período ─────────────────────────────────────────
    for period in tqdm(periods, desc="Calculating Period Indicators"):
        features[f'RSI_{period}'] = ta.RSI(stock_data['Close'], timeperiod=period)
        features[f'MFI_{period}'] = ta.MFI(stock_data['High'], stock_data['Low'], stock_data['Close'],
                                           stock_data['Volume'], timeperiod=period)
        features[f'ADX_{period}'] = ta.ADX(stock_data['High'], stock_data['Low'], stock_data['Close'],
                                           timeperiod=period)
        # Nota: OBV y AD no usan window; conservamos el sufijo para mantener tu esquema
        features[f'OBV_{period}'] = ta.OBV(stock_data['Close'], stock_data['Volume'])
        features[f'AD_{period}']  = ta.AD(stock_data['High'], stock_data['Low'],
                                          stock_data['Close'], stock_data['Volume'])
        features[f'ROCP_{period}'] = ta.ROCP(stock_data['Close'], timeperiod=period)

    # ── Series base (una sola vez) ─────────────────────────────────────
    log_ret = np.log(stock_data['Close']).diff()
    features['log_ret']       = log_ret
    features['log_ret_dif1']  = np.log(stock_data['Close']).diff(1)
    features['vola_10']       = log_ret.rolling(window=10, min_periods=10).std()
    features['autocorr_1']    = log_ret.rolling(window=20, min_periods=20)\
                                       .apply(lambda x: x.autocorr(lag=1), raw=False)
    features['log_t1']        = log_ret.shift(1)

    # ── Estimadores de volatilidad (se escriben en `features`) ─────────
    for i in tqdm(range(4, 20, 4), desc="Calculating Volatility Estimators"):
        features[f'parkinson_{i}']       = get_parkinson(stock_data,       window=i, trading_periods=50, clean=True)
        features[f'GermanKlass_{i}']     = get_GermanKlass(stock_data,     window=i, trading_periods=50, clean=True)
        features[f'RogersSatchell_{i}']  = get_RogersSatchell(stock_data,  window=i, trading_periods=50, clean=True)
        features[f'YangZhang_{i}']       = get_YangZhang(stock_data,       window=i, trading_periods=50, clean=True)
        features[f'HodgesTompkins_{i}']  = get_HodgesTompkins(stock_data,  window=i, trading_periods=50, clean=True)
        features[f'kurtosis_{i}']        = get_kurtosis(stock_data,        window=i, clean=True)
        features[f'skew_{i}']            = get_skew(stock_data,            window=i, clean=True)

    # ── Cruces SMA / EMA ───────────────────────────────────────────────
    for short_period in tqdm(short_periods, desc="Calculating SMA/EMA Crossovers"):
        for long_period in long_periods:
            features[f'SMA_Crossover_{short_period}_{long_period}'] = (
                ta.SMA(stock_data['Close'], timeperiod=short_period)
                - ta.SMA(stock_data['Close'], timeperiod=long_period)
            )
            features[f'EMA_Crossover_{short_period}_{long_period}'] = (
                ta.EMA(stock_data['Close'], timeperiod=short_period)
                - ta.EMA(stock_data['Close'], timeperiod=long_period)
            )

    # ── Kalman y derivados ─────────────────────────────────────────────
    for period in tqdm(kalman_periods, desc="Calculating Kalman and Derivatives"):
        kal = pd.Series(kalman_line(stock_data['Close'], kalman_length=period, smooth=3),
                        index=stock_data.index)
        features[f'Kal_{period}']              = kal
        features[f'Close_Kal_{period}_3']      = stock_data['Close'] - kal
        features[f'Kal_change_{period}_3']     = kal.diff()

    # ── Slopes ─────────────────────────────────────────────────────────
    for period in tqdm(kalman_periods, desc="Calculating Slopes"):
        for sLen in slope_length:
            df_s = slope(stock_data['Close'], length_kal=period, smooth_kal=3,
                         slopeLen=sLen, offset=-1)
            features[f'slope_div_{period}_{sLen}']             = df_s['slope_div']
            features[f'slope_signal_{period}_{sLen}']          = df_s['slope_signal']
            features[f'slope_angle_{period}_{sLen}']           = df_s['slope_angle']
            features[f'slope_angle_signal_{period}_{sLen}']    = df_s['slope_angle_signal']
            features[f'slope_lin_reg_{period}_{sLen}']         = df_s['slope_lin_reg']
            features[f'slope_lin_reg_signal_{period}_{sLen}']  = df_s['slope_lin_reg_signal']

    features.dropna(inplace=True)
    return features

# Support Functions

In [24]:

# Make sure SYMBOL exists (your code elsewhere uses SYMBOL, not `symbol`)
try:
    SYMBOL
except NameError:
    SYMBOL = "BTCUSD"   # fallback; change if needed

# Base folder where your models live
root_data = "/content/drive/MyDrive/Course Folder/Forex/XAUUSD/"
models_dir = Path(root_data) / "Models"

# Build full paths (use straight quotes and SYMBOL)
long_model_path  = models_dir / f"{SYMBOL}_Long_ml_model.joblib"
short_model_path = models_dir / f"{SYMBOL}_Short_ml_model.joblib"

# Helper to load and validate
def _load_joblib_model(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"Model file not found: {path}")
    model = joblib.load(path)
    return model

# Load globals used by main()
long_ml_model  = _load_joblib_model(long_model_path)
short_ml_model = _load_joblib_model(short_model_path)

print(f"✅ Long model loaded from:  {long_model_path}")
print(f"✅ Short model loaded from: {short_model_path}")
# ================================================================


EXPECTED_CLASSES = [0, 1, 2]

def _ensure_prob_cols_present(df: pd.DataFrame, side: str) -> None:
    """Guarantee prob_0/1/2_{side} exist so they are always saved to CSV."""
    for k in EXPECTED_CLASSES:
        col = f"prob_{k}_{side}"
        if col not in df.columns:
            df[col] = np.nan

def _reorder_ml(cols: list[str]) -> list[str]:
    cols = cols.copy()
    # unified first
    uni = ["label_ml", "prob_0", "prob_1", "prob_2"]
    for c in uni:
        if c in cols:
            cols.remove(c)
    insert_at = cols.index("Type") + 1 if "Type" in cols else len(cols)
    for c in uni:
        if c in df_out.columns:
            cols.insert(insert_at, c)
            insert_at += 1

    # then side-specific (if you keep them)
    ml_block = [
        "label_ml_long", "prob_0_long", "prob_1_long", "prob_2_long",
        "label_ml_short","prob_0_short","prob_1_short","prob_2_short"
    ]
    for c in ml_block:
        if c in cols:
            cols.remove(c)
    for c in ml_block:
        if c in df_out.columns:
            cols.insert(insert_at, c)
            insert_at += 1
    return cols

# ────────────────────────────────────────────────────────────────────
# FIRST-RUN INITIALIZER
# ────────────────────────────────────────────────────────────────────
async def initialize_csv_first_run(
    account,
    *,
    bars: int = 900,
    save_mode: str = "all",  # "all" = save full 900 rows; "last" = save only last row
    length_1: int,
    length_2: int,
    length_3: int,
    length_4: int,
    smooth_1: int,
    smooth_2: int,
    smooth_3: int,
    smooth_4: int,
):
    """
    Fetch `bars` candles, compute ATR + signals (+ ML if a trigger exists),
    and write either the full dataset or just the last row to the CSV.

    save_mode:
      - "all"  → write all rows (what you asked for)
      - "last" → compute with 900 but persist only the last bar
    """
    # 1) Download history
    df_hist = await get_candles_5m(account, start=None, limit=bars)
    df_hist = (df_hist
               .drop_duplicates("time")
               .sort_values("time")
               .reset_index(drop=True))

    # 2) ATR on all rows
    if len(df_hist) >= 14:
        df_hist["ATR"] = ta.ATR(df_hist["high"], df_hist["low"], df_hist["close"], 14).round(4)

    # 3) Signals (and ML only if an entry trigger exists)
    df_all = df_hist.copy()
    generate_trade_signals(
        df_all, length_1, length_2, length_3, length_4,
        smooth_1, smooth_2, smooth_3, smooth_4
    )
    maybe_compute_ml_for_entry(
        df_all,
        long_model=long_ml_model,
        short_model=short_ml_model,
        feature_cols_long=FEATURES_LONG,
        feature_cols_short=FEATURES_SHORT
    )
    sync_unified_ml_cols(df_all)
    _ensure_order_cols(df_all)

    # 4) Persist
    if save_mode == "all":
        df_all["source"] = 1
        stamp_system_time(df_all, mode="missing")   # put System_time on every row
        save_csv(df_all)
        print(f"✔ Archivo inicial creado: {len(df_all)} filas guardadas (se analizaron {bars} velas)")
    else:
        last = df_all.tail(1).copy()
        last["source"] = 1
        stamp_system_time(last, mode="last")
        save_csv(last)
        print(f"✔ Archivo inicial creado: 1 fila guardada (se analizaron {bars} velas)")



def has_new_entry_signal(df: pd.DataFrame) -> bool:
    """
    True only if the *last* bar is an entry trigger: Open_Trade ∈ {+1, -1}.
    """
    if df is None or df.empty or "Open_Trade" not in df.columns:
        return False
    s = df["Open_Trade"].iloc[-1]
    if pd.isna(s):
        return False
    return s in (1, -1)

def _ensure_unified_ml_cols(df: pd.DataFrame) -> None:
    """Make sure the 4 generic ML columns exist."""
    if "label_ml" not in df.columns:
        df["label_ml"] = np.nan
    for k in (0, 1, 2):
        c = f"prob_{k}"
        if c not in df.columns:
            df[c] = np.nan


def sync_unified_ml_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    Populate the 4 generic columns from side-specific ones:
      • If Open_Trade == +1  → copy from *_long
      • If Open_Trade == -1  → copy from *_short
      • Otherwise leave NaNs (no trade)
    Works for all rows that have side predictions.
    """
    if df is None or df.empty:
        return df

    _ensure_unified_ml_cols(df)

    # Prefer rows with explicit Open_Trade signal
    if "Open_Trade" in df.columns:
        # LONG rows
        mL = df["Open_Trade"] == 1
        if "label_ml_long" in df.columns:
            df.loc[mL, "label_ml"] = df.loc[mL, "label_ml_long"]
        for k in (0, 1, 2):
            src = f"prob_{k}_long"
            if src in df.columns:
                df.loc[mL, f"prob_{k}"] = df.loc[mL, src]

        # SHORT rows
        mS = df["Open_Trade"] == -1
        if "label_ml_short" in df.columns:
            df.loc[mS, "label_ml"] = df.loc[mS, "label_ml_short"]
        for k in (0, 1, 2):
            src = f"prob_{k}_short"
            if src in df.columns:
                df.loc[mS, f"prob_{k}"] = df.loc[mS, src]

    # Fallback: if some rows have predictions but Open_Trade is NaN, copy whichever exists
    if "label_ml" in df.columns:
        if "label_ml_long" in df.columns:
            mask = df["label_ml"].isna() & df["label_ml_long"].notna()
            df.loc[mask, "label_ml"] = df.loc[mask, "label_ml_long"]
            for k in (0, 1, 2):
                src = f"prob_{k}_long"
                if src in df.columns:
                    df.loc[mask, f"prob_{k}"] = df.loc[mask, src]

        if "label_ml_short" in df.columns:
            mask = df["label_ml"].isna() & df["label_ml_short"].notna()
            df.loc[mask, "label_ml"] = df.loc[mask, "label_ml_short"]
            for k in (0, 1, 2):
                src = f"prob_{k}_short"
                if src in df.columns:
                    df.loc[mask, f"prob_{k}"] = df.loc[mask, src]

    return df


def maybe_compute_ml_for_entry(
    df: pd.DataFrame,
    *,
    long_model,
    short_model,
    feature_cols_long: list[str] | None = None,
    feature_cols_short: list[str] | None = None
) -> pd.DataFrame:
    """
    Runs the heavy feature/ML block **only** when the last bar has an entry signal.
    Otherwise, it NOOPs (and ensures ML columns exist so downstream code never breaks).
    """
    # No entry trigger on the last bar → do nothing (but create empty ML cols)
    if not has_new_entry_signal(df):
        for side in ("long", "short"):
            lab = f"label_ml_{side}"
            if lab not in df.columns:
                df[lab] = np.nan
            for k in (0, 1, 2):
                col = f"prob_{k}_{side}"
                if col not in df.columns:
                    df[col] = np.nan
        return df

    # Entry trigger right now → run the real ML block (this computes features internally)
    return run_side_models_inplace(
        df,
        long_model=long_model,
        short_model=short_model,
        feature_cols_long=feature_cols_long,
        feature_cols_short=feature_cols_short
    )

def prediction_table_action(trade_side: int, pred: int):
    """
    Implements your table:
      trade=+1 (long): pred 0→None, 1→(tp1,sl1), 2→(tp2,sl2)
      trade=-1 (short): pred 0→None, 1→(tp1,sl1), 2→(tp2,sl2)
    Returns ("none"|"long"|"short", tp_mult, sl_mult).
    """
    if pred == 0 or not np.isfinite(pred):
        return ("none", None, None)
    if trade_side == 1:
        return ("long",  "takeprofit_1" if pred == 1 else "takeprofit_2",
                        "stoploss_1"   if pred == 1 else "stoploss_2")
    if trade_side == -1:
        return ("short", "takeprofit_1" if pred == 1 else "takeprofit_2",
                        "stoploss_1"   if pred == 1 else "stoploss_2")
    return ("none", None, None)



def has_new_entry_signal(df: pd.DataFrame) -> bool:
    """True only if the *last* bar is an entry trigger: Open_Trade ∈ {+1, -1}."""
    if df is None or df.empty or "Open_Trade" not in df.columns:
        return False
    s = df["Open_Trade"].iloc[-1]
    return np.isfinite(s) and (s == 1 or s == -1)

def _encoded_span_from(names: list[str]) -> int:
    """Return N if Encoded_0..Encoded_{N-1} appear in names; else 0."""
    if not names:
        return 0
    idxs = []
    for n in names:
        m = re.match(r"^Encoded_(\d+)$", str(n))
        if m:
            idxs.append(int(m.group(1)))
    return (max(idxs) + 1) if idxs else 0


# --- add this helper (near your other small utils) -------------------
def _preferred_feature_order(side: str) -> list[str]:
    """
    Return the exact column order to encode based on your files.
    side: 'long' or 'short'
    """
    if side == "long" and FEATURES_LONG:
        return FEATURES_LONG
    if side == "short" and FEATURES_SHORT:
        return FEATURES_SHORT
    return []


def _ensure_10m_prefix_aliases(X: pd.DataFrame, required_cols: list[str]) -> pd.DataFrame:
    """
    If your CSV expects '10_min_10_min_' but you produced '10min_10min_' (or vice-versa),
    make alias columns so required names exist without recomputing features.
    """
    if not required_cols:
        return X

    pref_a = "10min_10min_"
    pref_b = "10_min_10_min_"

    have = set(X.columns)
    for col in required_cols:
        if not isinstance(col, str):
            continue

        # CSV wants B, we have A → create B from A
        if col.startswith(pref_b) and col not in have:
            alt = pref_a + col[len(pref_b):]
            if alt in have:
                X[col] = X[alt]

        # CSV wants A, we have B → create A from B
        if col.startswith(pref_a) and col not in have:
            alt = pref_b + col[len(pref_a):]
            if alt in have:
                X[col] = X[alt]

    return X


def compute_10m_features_from_5m(df5: pd.DataFrame,
                                 *,
                                 prefix: str = "10min_10min_",
                                 pre_scale: bool = True) -> pd.DataFrame:
    """
    From a 5-minute OHLCV df (columns: time, open, high, low, close, volume, ...),
    resample to 10m, compute indicators via `create_features`, optionally scale them,
    prefix with `prefix`, then align back to the 5m time index (as-of, ffill).

    Returns a dataframe indexed by the 5m timestamps with only the prefixed columns.
    """
    if "time" in df5.columns:
        idx5 = pd.to_datetime(df5["time"], utc=True)
        base = df5.set_index(idx5)
    else:
        base = df5.copy()
        base.index = pd.to_datetime(base.index, utc=True)
        idx5 = base.index

    # robust lower/upper column map for resample
    cols = {c.lower(): c for c in base.columns}
    # minimal OHLCV present?
    for need in ("open", "high", "low", "close"):
        if need not in cols:
            raise ValueError(f"compute_10m_features_from_5m: missing column '{need}'")

    agg = {
        cols.get("open", "open"):  "first",
        cols.get("high", "high"):  "max",
        cols.get("low", "low"):    "min",
        cols.get("close", "close"): "last",
    }
    # volume-like fields are optional
    if "volume" in cols:
        agg[cols["volume"]] = "sum"
    if "tickvolume" in cols:
        agg[cols["tickvolume"]] = "sum"
    if "spread" in cols:
        agg[cols["spread"]] = "mean"

    ohlc10 = (base
              .resample("10T", label="right", closed="right")
              .agg(agg)
              .dropna(subset=[cols["open"], cols["high"], cols["low"], cols["close"]]))

    # TA-Lib expects Title-case OHLCV names
    block = ohlc10.rename(columns={
        cols.get("open", "open"):  "Open",
        cols.get("high", "high"):  "High",
        cols.get("low", "low"):    "Low",
        cols.get("close", "close"): "Close",
        cols.get("volume", "volume"): "Volume",
    })

    f10 = create_features(block)  # <- your function above

    if pre_scale:
        sc = StandardScaler()
        f10 = pd.DataFrame(sc.fit_transform(f10), index=f10.index, columns=f10.columns)

    # prefix
    f10.columns = [f"{prefix}{c}" for c in f10.columns]

    # align back to the 5m timestamps (as-of / forward fill)
    aligned = f10.reindex(idx5.union(f10.index)).ffill().reindex(idx5)
    return aligned


def append_10m_features_inplace(
    df_5m: pd.DataFrame,
    *,
    prefix: str = "10min_10min_",
    pre_scale: bool = True
) -> None:
    """
    Builds a 10-minute OHLCV from your 5m data, computes your `create_features`,
    optionally scales them, and appends to df_5m with the given prefix.

    Requirements:
      - df_5m has at least ['time','open','high','low','close','volume'].
      - `create_features(stock_data)` is defined and returns a DataFrame indexed by time.
    """
    if df_5m is None or df_5m.empty or "time" not in df_5m.columns:
        return

    # Ensure we won't create a column literally named 0 (from accidental index resets)
    df5 = df_5m.copy()
    t = pd.to_datetime(df5["time"], utc=True, errors="coerce")
    base = df5.set_index(t).sort_index()

    # Build 10m OHLCV
    agg = {
        "open": "first", "high": "max", "low": "min", "close": "last",
        "volume": "sum"
    }
    # optional columns if present
    if "tickVolume" in base.columns: agg["tickVolume"] = "sum"
    if "spread"     in base.columns: agg["spread"]     = "mean"

    ohlc10 = base.resample("10T").agg(agg).dropna(subset=["open","close"])

    # Map to Title-case columns for your create_features()
    stock10 = ohlc10.copy()
    stock10["Open"]   = stock10["open"]
    stock10["High"]   = stock10["high"]
    stock10["Low"]    = stock10["low"]
    stock10["Close"]  = stock10["close"]
    stock10["Volume"] = stock10["volume"]

    # Compute indicators with your existing function
    try:
        feats10 = create_features(stock10[["Open","High","Low","Close","Volume"]])
    except Exception as e:
        # If create_features fails, do nothing (but do not crash live loop)
        print(f"⚠️ 10m feature block skipped (create_features failed): {e}")
        return

    # Scale the 10m features if requested
    if pre_scale and not feats10.empty:
        scaler_10 = StandardScaler()
        feats10 = pd.DataFrame(
            scaler_10.fit_transform(feats10),
            index=feats10.index, columns=feats10.columns
        )

    # Bring the 10m features to 5m timestamps via asof/ffill
    feats10 = feats10.sort_index()
    # Align index (5m time index)
    idx5 = base.index
    feats10_5m = pd.merge_asof(
        pd.DataFrame(index=idx5).sort_index(),
        feats10.sort_index(),
        left_index=True, right_index=True, direction="backward"
    )

    # Prefix and append to original df_5m (in place)
    feats10_5m.columns = [prefix + str(c) for c in feats10_5m.columns]
    # Write back by 'time' (align on row order using the same index)
    for col in feats10_5m.columns:
        df_5m[col] = feats10_5m[col].values


# === Utilidades de columnas / guardado ===
def stamp_system_time(df: pd.DataFrame, mode: str = "last") -> None:
    """
    Sella System_time con la hora del sistema (UTC, sin milisegundos).
    mode="last": solo la última fila
    mode="missing": rellena donde esté NaT/NaN
    """
    if df is None or df.empty:
        return
    _ensure_order_cols(df)
    now_utc = pd.Timestamp.now(tz="UTC").floor("s")
    if mode == "last":
        df.at[df.index[-1], "System_time"] = now_utc
    else:  # "missing"
        mask = df["System_time"].isna()
        if mask.any():
            df.loc[mask, "System_time"] = now_utc

def _ensure_order_cols(df: pd.DataFrame) -> None:
    """Crea columnas con dtypes correctos; quita 'id' y 'actionType'. Incluye 'source', 'base_px', 'atr_base' y 'Real_SL'."""
    # Elimina columnas heredadas
    for col in ("id", "actionType"):
        if col in df.columns:
            df.drop(columns=[col], inplace=True)

    col_types = {
        "System_time":   "datetime64[ns, UTC]",
        "orderId":       "string",
        "magic":         "Int64",
        "symbol":        "string",
        "openPrice":     "float64",
        "comment":       "string",
        "Type":          "string",               # ← única columna de dirección
        "Entry_Date":    "datetime64[ns, UTC]",
        "Stop_Loss_atr": "float64",
        "Stop_Loss_$":   "float64",
        "Real_SL":       "float64",              # ← NUEVO
        "ATR":           "float64",
        "atr_mult_high": "float64",
        "atr_mult_low":  "float64",
        "trade_size":    "float64",
        "profits":       "float64",
        "base_px":       "float64",
        "atr_base":      "float64",
        "source":        "Int64",
    }

    for c, dtp in col_types.items():
        if c not in df.columns:
            if isinstance(dtp, str) and dtp.startswith("datetime64"):
                df[c] = pd.NaT
            elif dtp == "Int64":
                df[c] = pd.Series(pd.NA, dtype="Int64")
            elif dtp == "string":
                df[c] = pd.Series(pd.NA, dtype="string")
            else:
                df[c] = np.nan

    # normaliza sin romper datos existentes
    for c, dtp in col_types.items():
        try:
            if dtp == "string": df[c] = df[c].astype("string")
            if dtp == "Int64":  df[c] = df[c].astype("Int64")
        except Exception:
            pass

def save_csv(df: pd.DataFrame, path: str = FILE_PATH) -> None:
    """
    Reordena y guarda el CSV con formato estable:
      • System_time antes de 'time'
      • 'source' justo a la derecha de 'time' y antes de 'open'
      • Entry_Date justo ANTES de 'Stop_Loss_atr'
      • Real_SL a la derecha de 'Stop_Loss_$'; luego base_px y atr_base
      • Bloque ML (label/prob_*) inmediatamente después de 'Type'
    Formatea System_time/time/Entry_Date sin tz. Elimina 'id', 'brokerTime' y 'actionType' si aparecieran.
    """
    _ensure_order_cols(df)

    df_out = df.copy()

    # Quitar columnas heredadas si existen
    df_out.drop(columns=["id", "brokerTime", "actionType"], errors="ignore", inplace=True)

    # Formatear tiempos como strings sin tz
    for col in ["System_time", "time", "Entry_Date"]:
        if col in df_out.columns:
            ser = pd.to_datetime(df_out[col], errors="coerce", utc=True)
            df_out[col] = ser.dt.strftime("%Y-%m-%d %H:%M:%S")

    # ───────────────────── Helpers de reorden ─────────────────────
    def _reorder_for_entry_date(cols: list[str]) -> list[str]:
        if "Entry_Date" not in cols:
            return cols
        cols = cols.copy()
        cols.remove("Entry_Date")
        if "Stop_Loss_atr" in cols:
            cols.insert(cols.index("Stop_Loss_atr"), "Entry_Date")
        elif "Type" in cols:
            cols.insert(cols.index("Type") + 1, "Entry_Date")
        else:
            cols.append("Entry_Date")
        return cols

    def _reorder_stop_cols(cols: list[str]) -> list[str]:
        """Coloca Real_SL inmediatamente después de Stop_Loss_$; luego base_px y atr_base."""
        cols = cols.copy()
        # quitar para reinsertar
        for c in ["Real_SL", "base_px", "atr_base"]:
            if c in cols:
                cols.remove(c)
        if "Stop_Loss_$" in cols:
            i = cols.index("Stop_Loss_$") + 1
            for c in ["Real_SL", "base_px", "atr_base"]:
                if c in df_out.columns:
                    cols.insert(i, c)
                    i += 1
        else:
            for c in ["Real_SL", "base_px", "atr_base"]:
                if c in df_out.columns and c not in cols:
                    cols.append(c)
        return cols

    def _reorder_source(cols: list[str]) -> list[str]:
        """Coloca 'source' inmediatamente después de 'time' y antes de 'open' si aplica."""
        cols = cols.copy()
        if "source" in cols:
            cols.remove("source")
        if "time" in cols:
            i = cols.index("time") + 1
            cols.insert(i, "source")
            # si quedó después de 'open', muévelo antes
            if "open" in cols and cols.index("source") > cols.index("open"):
                cols.remove("source")
                cols.insert(cols.index("open"), "source")
        else:
            if "open" in cols:
                cols.insert(cols.index("open"), "source")
            else:
                cols.append("source")
        return cols

    def _reorder_ml(cols: list[str]) -> list[str]:
        """Coloca las columnas del modelo (predicción y probas) después de 'Type'."""
        cols = cols.copy()
        ml_block = [
            "label_ml_long", "prob_0_long", "prob_1_long", "prob_2_long",
            "label_ml_short", "prob_0_short", "prob_1_short", "prob_2_short"
        ]
        # eliminar del orden actual para reinsertarlas en bloque
        for c in ml_block:
            if c in cols:
                cols.remove(c)
        insert_at = cols.index("Type") + 1 if "Type" in cols else len(cols)
        for c in ml_block:
            if c in df_out.columns:
                cols.insert(insert_at, c)
                insert_at += 1
        return cols
    # ──────────────────────────────────────────────────────────────

    # Orden base (System_time inmediatamente antes de 'time')
    cols = list(df_out.columns)
    if "time" in cols:
        cols_wo_sys = [c for c in cols if c != "System_time"]
        i = cols_wo_sys.index("time")
        ordered = cols_wo_sys[:i] + ["System_time"] + cols_wo_sys[i:]
    else:
        ordered = cols

    # Aplicar reglas de reorden
    ordered = _reorder_source(ordered)
    ordered = _reorder_for_entry_date(ordered)
    ordered = _reorder_stop_cols(ordered)
    ordered = _reorder_ml(ordered)

    # Guardar
    df_out.to_csv(path, index=False, columns=ordered)

def migrate_csv_if_needed(path: str = FILE_PATH) -> None:
    """
    Migra CSV existente:
      • Elimina 'id', 'brokerTime' y 'actionType' si existen.
      • Agrega si faltan: 'profits','trade_size','base_px','atr_base','source','System_time','Real_SL'.
      • Formatea System_time/time/Entry_Date sin tz.
      • Reordena: System_time antes de time; source después de time y antes de open;
                  Entry_Date antes de Stop_Loss_atr;
                  Real_SL después de Stop_Loss_$; base_px/atr_base después de Real_SL.
    """
    if not os.path.exists(path):
        return

    df = pd.read_csv(path)

    df.drop(columns=["id", "brokerTime", "actionType"], errors="ignore")

    ensure_cols = {
        "System_time": pd.NaT,
        "profits":     np.nan,
        "trade_size":  np.nan,
        "base_px":     np.nan,
        "atr_base":    np.nan,
        "source":      pd.NA,
        "Real_SL":     np.nan,  # ← NUEVO
    }
    for c, default in ensure_cols.items():
        if c not in df.columns:
            df[c] = default

    for col in ["System_time", "time", "Entry_Date"]:
        if col in df.columns:
            ser = pd.to_datetime(df[col], errors="coerce", utc=True)
            df[col] = ser.dt.strftime("%Y-%m-%d %H:%M:%S")

    def _reorder_for_entry_date(cols: list[str]) -> list[str]:
        if "Entry_Date" not in cols: return cols
        cols = cols.copy()
        cols.remove("Entry_Date")
        if "Stop_Loss_atr" in cols:
            cols.insert(cols.index("Stop_Loss_atr"), "Entry_Date")
        elif "Type" in cols:
            cols.insert(cols.index("Type") + 1, "Entry_Date")
        else:
            cols.append("Entry_Date")
        return cols

    def _reorder_stop_cols(cols: list[str]) -> list[str]:
        cols = cols.copy()
        for c in ["Real_SL", "base_px", "atr_base"]:
            if c in cols: cols.remove(c)
        if "Stop_Loss_$" in cols:
            i = cols.index("Stop_Loss_$") + 1
            if "Real_SL" in df.columns: cols.insert(i, "Real_SL"); i += 1
            for c in ["base_px", "atr_base"]:
                if c in df.columns:
                    cols.insert(i, c); i += 1
        else:
            for c in ["Real_SL", "base_px", "atr_base"]:
                if c in df.columns and c not in cols: cols.append(c)
        return cols

    def _reorder_source(cols: list[str]) -> list[str]:
        cols = cols.copy()
        if "source" in cols: cols.remove("source")
        if "time" in cols:
            i = cols.index("time") + 1
            cols.insert(i, "source")
            if "open" in cols and cols.index("source") > cols.index("open"):
                cols.remove("source")
                cols.insert(cols.index("open"), "source")
        else:
            if "open" in cols: cols.insert(cols.index("open"), "source")
            else: cols.append("source")
        return cols

    cols = list(df.columns)
    if "time" in cols:
        cols_wo_sys = [c for c in cols if c != "System_time"]
        i = cols_wo_sys.index("time")
        ordered = cols_wo_sys[:i] + ["System_time"] + cols_wo_sys[i:]
    else:
        ordered = cols

    ordered = _reorder_source(ordered)
    ordered = _reorder_for_entry_date(ordered)
    ordered = _reorder_stop_cols(ordered)

    df.to_csv(path, index=False, columns=ordered)

def _load_csv() -> pd.DataFrame:
    """Lee el CSV preservando tipos; convierte tiempos a UTC tz-aware; elimina 'id', 'actionType' y 'brokerTime'."""
    if not os.path.exists(FILE_PATH):
        return pd.DataFrame()

    df = pd.read_csv(
        FILE_PATH,
        dtype={
            "orderId": "string",
            "symbol":  "string",
            "comment": "string",
            "Type":    "string",
        }
    )

    df.drop(columns=["id", "brokerTime", "actionType"], errors="ignore")

    for col in ["time", "Entry_Date", "System_time"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)

    return df

def stamp_system_time(df: pd.DataFrame, mode: str = "last") -> None:
    """
    Sella System_time con hora del sistema (UTC, sin milisegundos).
    mode="last": solo la última fila; mode="missing": rellena las que estén vacías.
    """
    if df is None or df.empty:
        return
    _ensure_order_cols(df)
    now_utc = pd.Timestamp.now(tz="UTC").floor("s")
    if mode == "last":
        df.at[df.index[-1], "System_time"] = now_utc
    else:
        mask = df["System_time"].isna()
        if mask.any():
            df.loc[mask, "System_time"] = now_utc

async def get_current_candle_snapshot(account,
                                      rpc_conn,
                                      symbol: str = SYMBOL,
                                      timeframe: str = time_frame_data) -> pd.DataFrame:
    """
    Devuelve 1 fila con la vela más reciente (puede ser la vela en curso)
    con campos: time, open, high, low, close, volume, tickVolume, spread.

    Intenta primero vía RPC (candles en vivo) y, si falla, usa el
    endpoint histórico como respaldo.
    """
    def _to_row(c: dict) -> dict:
        return {
            "time":       pd.to_datetime(c.get("time"), utc=True, errors="coerce"),
            "open":       float(c.get("open"))       if c.get("open")       is not None else np.nan,
            "high":       float(c.get("high"))       if c.get("high")       is not None else np.nan,
            "low":        float(c.get("low"))        if c.get("low")        is not None else np.nan,
            "close":      float(c.get("close"))      if c.get("close")      is not None else np.nan,
            "volume":     float(c.get("volume"))     if c.get("volume")     is not None else np.nan,
            "tickVolume": float(c.get("tickVolume") if c.get("tickVolume") is not None else c.get("tick_volume") or np.nan),
            "spread":     float(c.get("spread"))     if c.get("spread")     is not None else np.nan,
        }

    # 1) Intento RPC (normalmente expone la vela actual)
    try:
        # Variantes posibles según SDK; probamos dos firmas comunes.
        try:
            # a) límite por cantidad
            candles = await rpc_conn.get_candles(symbol=SYMBOL, timeframe=timeframe, limit=1)
        except TypeError:
            # b) rango por tiempo (últimos minutos); tomamos la última
            to_ts   = dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc)
            from_ts = to_ts - dt.timedelta(minutes=10)
            candles = await rpc_conn.get_candles(symbol=SYMBOL, timeframe=timeframe,
                                                 start_time=from_ts, end_time=to_ts)
        if candles:
            row = _to_row(candles[-1])
            return pd.DataFrame([row])
    except Exception:
        pass

    # 2) Respaldo: histórico (toma la más reciente cerrada si el broker no sirve la actual)
    try:
        candles = await account.get_historical_candles(symbol=SYMBOL, timeframe=timeframe, start_time=None, limit=1)
        if candles:
            row = _to_row(candles[-1])
            return pd.DataFrame([row])
    except Exception:
        pass

    # 3) Si todo falla, devolver DF vacío con columnas estandarizadas
    return pd.DataFrame(columns=["time", "open", "high", "low", "close", "volume", "tickVolume", "spread"])


###############################################################################
# CONFIGURACIÓN GENERAL

logging.basicConfig(level=logging.ERROR)
logging.getLogger("metaapi_cloud_sdk").setLevel(logging.ERROR)
logging.getLogger("websockets").setLevel(logging.ERROR)

#from metaapi_cloud_sdk.clients.error_handling import TradeException

MetaApi.enable_logging()
logging.basicConfig(level=logging.ERROR)
for noisy in ("metaapi_cloud_sdk", "socketio", "engineio"):
    logging.getLogger(noisy).setLevel(logging.ERROR)

###############################################################################
# CONEXIÓN A METAAPI

async def connect_metaapi(token: str, account_id: str):
    api     = MetaApi(token)
    account = await api.metatrader_account_api.get_account(account_id)
    conn    = account.get_rpc_connection()
    await conn.connect(); await conn.wait_synchronized()
    return account

###############################################################################
# DESCARGA DE VELAS

async def get_candles_5m(account, start: dt.datetime | None, limit: int = CANDEL_NUMBER):
    candles = await account.get_historical_candles(
        symbol=SYMBOL, timeframe= time_frame_data, start_time=start, limit = limit)
    return pd.DataFrame([{
        "time": pd.to_datetime(c["time"], utc=True),
        "open": c["open"], "high": c["high"], "low": c["low"], "close": c["close"],
        "volume": c["volume"], "tickVolume": c["tickVolume"], "spread": c["spread"]
    } for c in candles])

###############################################################################
# SINCRONIZAR CON EL RELOJ (próximo minuto:07)

def seconds_until_next_m07() -> float:
    now = dt.datetime.utcnow()
    nxt = (now + dt.timedelta(minutes=1)).replace(second=3, microsecond=0)
    return max((nxt - now).total_seconds(), 0.5)

def seconds_until_next_5m03() -> float:
    """
    Seconds until the next 5-minute boundary + 3s (to let the bar close on most feeds).
    Triggers at hh:00:03, :05:03, :10:03, ... UTC.
    """
    now = dt.datetime.utcnow()
    # floor to current 5-minute block start
    start_5m = now.replace(minute=(now.minute // 5) * 5, second=0, microsecond=0)
    nxt = start_5m + dt.timedelta(minutes=5, seconds=3)
    return max((nxt - now).total_seconds(), 0.5)


def generate_trade_signals(
    df: pd.DataFrame,
    length_1: int,
    length_2: int,
    length_3: int,
    length_4: int,
    smooth_1: int,
    smooth_2: int,
    smooth_3: int,
    smooth_4: int
) -> pd.DataFrame:
    """
    Calcula 4 líneas de Kalman sobre 'close' y crea:
      • kal_1, kal_2, kal_3, kal_4
      • Open_Trade: +1 (BUY) / -1 (SELL) cuando CAMBIA sesgo (k1..k3)
      • Close_Trade: -1 si kal_4 < kal_4.shift()  → cierra BUY
                     +1 si kal_4 > kal_4.shift()  → cierra SELL
        (señales consecutivas iguales se deduplican)
    """
    if df is None or df.empty:
        return df
    if "close" not in df.columns:
        raise ValueError("generate_trade_signals: falta columna 'close'.")

    close = pd.to_numeric(df["close"], errors="coerce").fillna(method="ffill")

    def _clamp_int(x, mn=1):
        try: x = int(x)
        except Exception: x = mn
        return max(x, mn)

    length_1 = _clamp_int(length_1); length_2 = _clamp_int(length_2)
    length_3 = _clamp_int(length_3); length_4 = _clamp_int(length_4)
    smooth_1 = _clamp_int(smooth_1); smooth_2 = _clamp_int(smooth_2)
    smooth_3 = _clamp_int(smooth_3); smooth_4 = _clamp_int(smooth_4)

    # Kalman lines
    df["kal_1"] = kalman_line(close, length_1, smooth_1)
    df["kal_2"] = kalman_line(close, length_2, smooth_2)
    df["kal_3"] = kalman_line(close, length_3, smooth_3)
    df["kal_4"] = kalman_line(close, length_4, smooth_4)

    # Sesgo por k1..k3
    k1_up, k2_up, k3_up = df["kal_1"] > df["kal_1"].shift(1), df["kal_2"] > df["kal_2"].shift(1), df["kal_3"] > df["kal_3"].shift(1)
    k1_dn, k2_dn, k3_dn = df["kal_1"] < df["kal_1"].shift(1), df["kal_2"] < df["kal_2"].shift(1), df["kal_3"] < df["kal_3"].shift(1)

    bull = k1_up & k2_up & k3_up
    bear = k1_dn & k2_dn & k3_dn
    aux  = np.where(bull, 1, np.where(bear, -1, np.nan))
    df["Open_Trade"] = np.where(pd.Series(aux).shift(1) != aux, aux, np.nan)

    # Cierre estrictamente por pendiente de kal_4
    k4_up = df["kal_4"] > df["kal_4"].shift(1)   # +1 → cierra SELL
    k4_dn = df["kal_4"] < df["kal_4"].shift(1)   # -1 → cierra BUY
    close_raw  = np.where(k4_dn, -1, np.where(k4_up, 1, np.nan))
    close_sr   = pd.Series(close_raw)
    df["Close_Trade"] = close_sr.where(close_sr != close_sr.shift(1), np.nan)

    return df

def _rest_place_order(auth_token: str,
                      account_id: str,
                      region: str,
                      symbol: str,
                      side: str,
                      volume: float,
                      comment: str = "Kal",
                      magic: int | None = None,
                      stop_loss: float | None = None,
                      take_profit: float | None = None,
                      timeout: int = 20):
    side = side.upper().strip()
    action_map = {"BUY": "ORDER_TYPE_BUY", "SELL": "ORDER_TYPE_SELL"}
    if side not in action_map:
        raise ValueError("side must be 'BUY' or 'SELL'")

    url = f"https://mt-client-api-v1.{region}.agiliumtrade.ai/users/current/accounts/{account_id}/trade"
    payload = {
        "symbol": symbol,
        "actionType": action_map[side],
        "volume": float(volume),
        "comment": str(comment)
    }
    if magic is not None:
        try: payload["magic"] = int(magic)
        except Exception: pass
    if stop_loss is not None:
        try: payload["stopLoss"] = float(stop_loss)
        except Exception: pass
    if take_profit is not None:
        try: payload["takeProfit"] = float(take_profit)
        except Exception: pass

    headers = {"Content-Type": "application/json", "Accept": "application/json", "auth-token": auth_token}
    return requests.post(url, headers=headers, json=payload, timeout=timeout)


# --- FIX 1: really use the symbol param here -------------------------------
def _rest_get_positions(auth_token: str,
                        account_id: str,
                        region: str,
                        symbol: str | None = None,
                        timeout: int = 15):
    url = f"https://mt-client-api-v1.{region}.agiliumtrade.ai/users/current/accounts/{account_id}/positions"
    headers = {"Accept": "application/json", "auth-token": auth_token}
    params = {}
    if symbol:
        params["symbol"] = str(symbol)   # << was using the global SYMBOL
    try:
        return requests.get(url, headers=headers, params=params, timeout=timeout)
    except Exception as e:
        class _Dummy:
            status_code = 0
            def json(self): return {"error": str(e)}
            text = str(e)
        return _Dummy()


def _rest_modify_position(auth_token: str,
                          account_id: str,
                          region: str,
                          position_id: str,
                          stop_loss: float | None = None,
                          take_profit: float | None = None,
                          timeout: int = 20):
    """
    Modifica una posición abierta (SL/TP) usando el endpoint REST de MetaApi.
    """
    url = f"https://mt-client-api-v1.{region}.agiliumtrade.ai/users/current/accounts/{account_id}/trade"
    payload = {"actionType": "POSITION_MODIFY", "positionId": str(position_id)}

    if stop_loss is not None:
        payload["stopLoss"] = float(stop_loss)

    if take_profit is not None:
        payload["takeProfit"] = float(take_profit)

    headers = {"Content-Type": "application/json", "Accept": "application/json", "auth-token": auth_token}

    return requests.post(url, headers=headers, json=payload, timeout=timeout)

# === DROP-IN REPLACEMENT ==============================================
async def open_trade(df: pd.DataFrame,
                     rpc_conn,
                     symbol: str = "BTCUSD",
                     lot: float = 1.0,
                     comment: str = "Kal",
                     magic: int = 900002):
    """
    Entra solo si:
      • Open_Trade ∈ {+1, -1}
      • y la predicción del modelo del lado (label_ml_long/label_ml_short) ∈ {1, 2}

    SL/TP al abrir (en precio) se calculan con ATR del último bar:
      • Para BUY:  SL = close - ATR * sl_mult ; TP = close + ATR * tp_mult
      • Para SELL: SL = close + ATR * sl_mult ; TP = close - ATR * tp_mult

    Donde (sl_mult, tp_mult) dependen de la clase:
      • class=1 → (stoploss_1, takeprofit_1)
      • class=2 → (stoploss_2, takeprofit_2)
      • class=0 → no abre
    """
    import re

    if df.empty or "Open_Trade" not in df.columns:
        return

    _ensure_order_cols(df)

    # ---------- helpers for existing open/magic sync ----------
    def _has_magic(p) -> bool:
        pm = p.get("magic", None)
        if pm is not None:
            try:
                if int(pm) == int(magic): return True
            except Exception:
                pass
        return f"magic={magic}" in str(p.get("comment") or "")

    def _side_of(p) -> str:
        t = p.get("type")
        if isinstance(t, str):
            tt = t.upper()
            if "BUY"  in tt: return "BUY"
            if "SELL" in tt: return "SELL"
        if t == 0: return "BUY"
        if t == 1: return "SELL"
        return ""

    def _split_comment_magic(cmt: str) -> tuple[str, int | None]:
        if not cmt: return "", None
        m = re.search(r"magic\s*=\s*(\d+)", cmt, flags=re.IGNORECASE)
        mag = int(m.group(1)) if m else None
        clean = cmt.split("|", 1)[0].strip()
        return clean, mag

    # ---------- already open? just sync and exit ----------
    try:
        positions = await _pull_positions_all_sources(rpc_conn, symbol)
    except Exception:
        positions = []
    open_with_magic = [p for p in positions if _has_magic(p)]

    now_utc = pd.Timestamp.now(tz="UTC").floor("s")
    row = df.index[-1]

    if open_with_magic:
        p = open_with_magic[0]
        df.at[row, "System_time"] = now_utc
        pm = p.get("magic")
        pc = str(p.get("comment") or "")
        clean_cmt, mag_from_cmt = _split_comment_magic(pc)

        if pm is not None and str(pm).strip() != "":
            try: df.at[row, "magic"] = int(pm)
            except Exception: df.at[row, "magic"] = int(magic)
        elif mag_from_cmt is not None:
            df.at[row, "magic"] = int(mag_from_cmt)
        else:
            df.at[row, "magic"] = int(magic)

        df.at[row, "symbol"]    = str(p.get("symbol") or symbol)
        df.at[row, "openPrice"] = float(p.get("openPrice") or p.get("price") or np.nan)
        df.at[row, "comment"]   = clean_cmt or str(comment)

        vol = p.get("volume") or p.get("lots") or None
        if vol is not None:
            try: df.at[row, "trade_size"] = float(vol)
            except Exception: pass

        side = _side_of(p)
        if side: df.at[row, "Type"] = "Long" if side == "BUY" else "Short"
        if pd.isna(df.at[row, "Entry_Date"]): df.at[row, "Entry_Date"] = now_utc

        save_csv(df)
        print("ℹ Position already open; synced last row and skipped new order.")
        return

    # ---------- side signal from Open_Trade ----------
    sig = df["Open_Trade"].iloc[-1]
    if not np.isfinite(sig):
        return
    side_req = "BUY" if sig == 1 else ("SELL" if sig == -1 else None)
    if side_req is None:
        return

    # ---------- read prediction for that side (0/1/2) ----------
    pred_col = "label_ml_long" if sig == 1 else "label_ml_short"
    pred_val = df[pred_col].iloc[-1] if pred_col in df.columns else np.nan
    try:
        pred_cls = int(pred_val)
    except Exception:
        pred_cls = np.nan

    # Rule: class 0 → do not place any order
    if not np.isfinite(pred_cls) or pred_cls == 0:
        print(f"ℹ No entry: {pred_col}={pred_val} → skip order.")
        return

    # Choose ATR multiples from prediction
    if pred_cls == 1:
        sl_mult, tp_mult = float(stoploss_1), float(takeprofit_1)
    elif pred_cls == 2:
        sl_mult, tp_mult = float(stoploss_2), float(takeprofit_2)
    else:
        print(f"ℹ Unknown class {pred_cls}; skip order.")
        return

    # ---------- prices from ATR ----------
    prev_close = float(df["close"].iloc[-1]) if pd.notna(df["close"].iloc[-1]) else np.nan
    atr_val    = float(df["ATR"].iloc[-1])   if "ATR" in df.columns and pd.notna(df["ATR"].iloc[-1]) else np.nan
    if not (np.isfinite(prev_close) and np.isfinite(atr_val)):
        print("✘ Missing close/ATR → skip order.")
        return

    if side_req == "BUY":
        sl_to_send = prev_close - atr_val * sl_mult
        tp_to_send = prev_close + atr_val * tp_mult
    else:  # SELL
        sl_to_send = prev_close + atr_val * sl_mult
        tp_to_send = prev_close - atr_val * tp_mult

    # ---------- send order with SL & TP ----------
    loop = asyncio.get_running_loop()
    resp = await loop.run_in_executor(
        None,
        lambda: _rest_place_order(
            auth_token=META_API_TOKEN,
            account_id=ACCOUNT_ID,
            region=REGION,
            symbol=symbol,
            side=side_req,
            volume=float(lot),
            comment=str(comment),
            magic=int(magic),
            stop_loss=sl_to_send,
            take_profit=tp_to_send,
            timeout=20
        )
    )
    if resp.status_code != 200:
        try: err = resp.json()
        except Exception: err = {"raw": resp.text[:500]}
        print("✘ Order failed", resp.status_code, json.dumps(err, indent=2, ensure_ascii=False))
        return

    data        = resp.json()
    order_id    = str(data.get("orderId") or "")
    position_id = str(data.get("positionId") or "")

    # ---------- write metadata ----------
    df.at[row, "System_time"]   = now_utc
    df.at[row, "orderId"]       = order_id
    df.at[row, "magic"]         = int(magic)
    df.at[row, "symbol"]        = symbol
    df.at[row, "comment"]       = str(comment)
    df.at[row, "Entry_Date"]    = now_utc
    df.at[row, "trade_size"]    = float(lot)
    df.at[row, "Stop_Loss_$"]   = float(sl_to_send)
    df.at[row, "Stop_Loss_atr"] = float(sl_mult)
    # (opcional) conservar TP en el CSV para trazabilidad
    try: df.at[row, "Take_Profit_$"] = float(tp_to_send)
    except Exception: pass

    # Try to fetch openPrice & type
    open_price, fetched_typ = np.nan, None
    if rpc_conn:
        try:
            for _ in range(60):
                pos_list = await _pull_positions_all_sources(rpc_conn, symbol)
                match = None
                for p in (pos_list or []):
                    pid = str(p.get("id") or p.get("positionId") or "")
                    if pid == position_id or pid == order_id or _has_magic(p):
                        match = p; break
                if match:
                    val = match.get("openPrice") or match.get("price") or match.get("open_price")
                    if val is not None: open_price = float(val)
                    t = _side_of(match)
                    if t: fetched_typ = "Long" if t == "BUY" else "Short"
                    break
                await asyncio.sleep(0.5)
        except Exception:
            pass

    if not np.isfinite(open_price):
        try: open_price = float(df.at[row, "close"])
        except Exception: open_price = np.nan
    df.at[row, "openPrice"] = open_price
    df.at[row, "Type"]      = fetched_typ if fetched_typ else ("Long" if side_req == "BUY" else "Short")

    save_csv(df)
    print(f"✅ {side_req} placed | cls={pred_cls} | SL@{sl_to_send:.2f} (x{sl_mult}) | "
          f"TP@{tp_to_send:.2f} (x{tp_mult}) | orderId={order_id} positionId={position_id}")


def atr_close(df: pd.DataFrame) -> pd.DataFrame:
    """
    Por bloque de trade:
      • forward-fill de metadatos (incluye Type)
      • fija 'base_px' y 'atr_base' con los valores de APERTURA del trade (constantes en el bloque)
      • calcula atr_mult_* usando ese 'atr_base'
      • calcula 'profits' SOLO si está NaN (para no pisar el de la API)
      • propaga Real_SL en el bloque con el último valor no nulo
    """
    if df.empty:
        return df

    for c in ('time', 'Entry_Date'):
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce', utc=True)

    # Reiniciamos solo columnas derivadas de ATR (no profits)
    for c in ('atr_mult_high', 'atr_mult_low'):
        if c not in df.columns: df[c] = np.nan
        else: df[c].values[:] = np.nan
    if 'profits' not in df.columns:
        df['profits'] = np.nan
    if 'base_px'  not in df.columns: df['base_px']  = np.nan
    if 'atr_base' not in df.columns: df['atr_base'] = np.nan
    if 'Real_SL'  not in df.columns: df['Real_SL']  = np.nan

    # Detectar inicios de trade (por orderId; fallback Entry_Date)
    starts_mask = pd.Series(False, index=df.index)
    if 'orderId' in df.columns:
        oid = df['orderId']
        starts_mask = oid.notna() & (oid != oid.shift(1))
    if not starts_mask.any() and 'Entry_Date' in df.columns:
        ed = pd.to_datetime(df['Entry_Date'], errors='coerce', utc=True)
        starts_mask = ed.notna() & (ed != ed.shift(1))
    if not starts_mask.any():
        return df

    groups = starts_mask.cumsum()
    trade_ids = groups[starts_mask].unique()

    meta_cols = ["orderId","magic","symbol","openPrice","comment","Type","Entry_Date","trade_size"]

    for gid in trade_ids:
        mask = (groups == gid)
        start_idx = df.index[mask][0]

        base_px = df.at[start_idx, 'openPrice'] if 'openPrice' in df.columns else np.nan
        try: base_px = float(base_px)
        except Exception: base_px = np.nan
        if not np.isfinite(base_px) and 'close' in df.columns:
            try: base_px = float(df.at[start_idx, 'close'])
            except Exception: base_px = np.nan

        atr_base = np.nan
        if 'atr_base' in df.columns and pd.notna(df.at[start_idx, 'atr_base']):
            try: atr_base = float(df.at[start_idx, 'atr_base'])
            except Exception: atr_base = np.nan
        if not np.isfinite(atr_base) and 'ATR' in df.columns and pd.notna(df.at[start_idx, 'ATR']):
            try: atr_base = float(df.at[start_idx, 'ATR'])
            except Exception: atr_base = np.nan

        typ = str(df.at[start_idx, 'Type']) if 'Type' in df.columns and pd.notna(df.at[start_idx, 'Type']) else None

        # Forward-fill de metadatos del bloque
        df.loc[mask, [c for c in meta_cols if c in df.columns]] = df.loc[start_idx, [c for c in meta_cols if c in df.columns]].values

        if np.isfinite(base_px):  df.loc[mask, 'base_px']  = base_px
        if np.isfinite(atr_base): df.loc[mask, 'atr_base'] = atr_base

        if np.isfinite(base_px) and np.isfinite(atr_base) and atr_base != 0.0 and typ in ('Long','Short'):
            if typ == 'Long':
                df.loc[mask, 'atr_mult_high'] = ((df.loc[mask, 'high'] - base_px) / atr_base).round(2)
                df.loc[mask, 'atr_mult_low']  = ((df.loc[mask,  'low'] - base_px) / atr_base).round(2)
            else:
                df.loc[mask, 'atr_mult_high'] = ((base_px - df.loc[mask, 'high']) / atr_base).round(2)
                df.loc[mask, 'atr_mult_low']  = ((base_px - df.loc[mask,  'low']) / atr_base).round(2)

        # Solo rellenar profits donde esté NaN (API manda en sync_stop_loss_from_df)
        size = float(df.at[start_idx, 'trade_size']) if 'trade_size' in df.columns and pd.notna(df.at[start_idx, 'trade_size']) else np.nan
        if np.isfinite(base_px) and np.isfinite(size) and typ in ('Long','Short'):
            m_nan = mask & df['profits'].isna()
            if m_nan.any():
                if typ == 'Long':
                    df.loc[m_nan, 'profits'] = ((df.loc[m_nan, 'close'] - base_px) * size).round(2)
                else:
                    df.loc[m_nan, 'profits'] = ((base_px - df.loc[m_nan, 'close']) * size).round(2)

    # === Propagar Real_SL dentro de cada bloque (último valor no nulo) =======
    if 'orderId' in df.columns and df['orderId'].notna().any():
        starts = df['orderId'].notna() & (df['orderId'] != df['orderId'].shift(1))
    else:
        ed2 = pd.to_datetime(df.get('Entry_Date'), errors='coerce', utc=True)
        starts = ed2.notna() & (ed2 != ed2.shift(1))

    if starts.any():
        grp = starts.cumsum()
        for gid in grp[starts].unique():
            m = (grp == gid)
            ser = df.loc[m, 'Real_SL']
            if ser.notna().any():
                val = float(np.round(ser.dropna().iloc[-1], 2))
                df.loc[m, 'Real_SL'] = val

    return df

def tick_dyn_atr(df: pd.DataFrame,
                 initial_atr: float = INITIAL_SL,
                 first_step_atr: float = FIRST_STEP_ATR,
                 gap_first_step_atr: float = GAP_FIRST_STEP_ATR) -> pd.DataFrame:
    """
    Dinámica de stop en múltiplos de ATR usando atr_mult_high/atr_mult_low.

    Además de calcular la columna 'tick_dyn_atr' (múltiplos ATR),
    **escribe el precio del stop en 'Stop_Loss_$'** a partir de:
      • base_px
      • atr_base
      • Type ('Long' o 'Short')
      • tick_dyn_atr (múltiplos)

    Fórmulas:
      Long  -> Stop_Loss_$ = base_px + atr_base * tick_dyn_atr
      Short -> Stop_Loss_$ = base_px - atr_base * tick_dyn_atr
    """
    col_name = 'tick_dyn_atr'
    if col_name not in df.columns:
        df[col_name] = np.nan
    if 'Stop_Loss_$' not in df.columns:
        df['Stop_Loss_$'] = np.nan
    # (opcional) mantener también el múltiplo en Stop_Loss_atr si existe esa columna
    if 'Stop_Loss_atr' not in df.columns:
        df['Stop_Loss_atr'] = np.nan

    in_trade       = False
    trade_active   = False
    broken         = False
    sl_val         = initial_atr
    next_threshold = first_step_atr

    for idx, row in df.iterrows():

        # Detecta nueva apertura real de trade
        new_open = (
            (('orderId'   in df.columns) and pd.notna(row.get('orderId'))) or
            (('openPrice' in df.columns) and pd.notna(row.get('openPrice'))) or
            (pd.notna(row.get('Entry_Date')))
        )

        if new_open and not in_trade:
            in_trade       = True
            trade_active   = True
            broken         = False
            sl_val         = initial_atr
            next_threshold = first_step_atr
            prev_sl        = sl_val

            # Alinear Entry_Date si fuera necesario (usamos 'time')
            entry_dt = row.get('time')
            if entry_dt is not None:
                df.at[idx, 'Entry_Date'] = entry_dt

        if in_trade:
            m_high = row.get('atr_mult_high', np.nan)
            m_low  = row.get('atr_mult_low',  np.nan)
            best_pnl = np.nanmax([m_high, m_low])
            best_pnl = 0.0 if np.isnan(best_pnl) else float(best_pnl)

            if trade_active and not broken:
                # subir el múltiplo del stop cada vez que cruzamos un umbral
                while best_pnl >= next_threshold:
                    sl_val         += gap_first_step_atr
                    next_threshold += gap_first_step_atr

                # si volvemos por debajo del último SL, marcamos roto
                below_prev = (
                    (np.isfinite(m_high) and m_high < prev_sl) or
                    (np.isfinite(m_low)  and m_low  < prev_sl)
                )
                if below_prev:
                    broken       = True
                    trade_active = False
                    in_trade     = False

            df.at[idx, col_name]      = np.nan if broken else sl_val
            df.at[idx, 'Stop_Loss_atr'] = np.nan if broken else sl_val
            prev_sl = sl_val

    # ── Convertir múltiplos a precio y guardarlo en Stop_Loss_$ ─────────────
    # Requiere: base_px, atr_base, Type y tick_dyn_atr válidos
    try:
        sl_mult = df[col_name].astype(float)
        base    = df['base_px'].astype(float)
        atr     = df['atr_base'].astype(float)
        typ     = df['Type'].astype('string')

        # precio = base ± atr * múltiplos (según dirección)
        stop_price = np.where(
            (typ == 'Long')  & np.isfinite(sl_mult) & np.isfinite(base) & np.isfinite(atr),
            base + atr * sl_mult,
            np.where(
                (typ == 'Short') & np.isfinite(sl_mult) & np.isfinite(base) & np.isfinite(atr),
                base - atr * sl_mult,
                np.nan
            )
        )

        # Redondeo suave (mantén o ajusta si lo prefieres con más decimales)
        df['Stop_Loss_$'] = pd.Series(stop_price, index=df.index).round(2)
    except Exception:
        # si faltan columnas o tipos, no rompemos el flujo
        pass

    return df

# --- REST fallback to close a position --------------------------------------
def _rest_close_position(auth_token: str,
                         account_id: str,
                         region: str,
                         position_id: str,
                         timeout: int = 20):
    """
    Cierra una posición por REST. En MetaApi el actionType es POSITION_CLOSE_ID.
    """
    url = f"https://mt-client-api-v1.{region}.agiliumtrade.ai/users/current/accounts/{account_id}/trade"
    payload = {"actionType": "POSITION_CLOSE_ID", "positionId": str(position_id)}
    headers = {"Content-Type": "application/json", "Accept": "application/json", "auth-token": auth_token}
    return requests.post(url, headers=headers, json=payload, timeout=timeout)


# --- Pull positions: fast attempts via RPC (short timeout) then REST --------
async def _pull_positions_all_sources(rpc_conn, symbol: str | None):
    """
    Positions with short RPC timeouts + REST fallback.
    Avoids hangs when websocket subscribe is flaky in Colab.
    """
    import asyncio
    positions = []

    async def _rpc_try(call, *args, **kwargs):
        try:
            return await asyncio.wait_for(call(*args, **kwargs), timeout=4)
        except Exception:
            return []

    # 1) RPC (with symbol), then RPC (all)
    if rpc_conn:
        positions = await _rpc_try(rpc_conn.get_positions, symbol=symbol)
        if not positions:
            positions = await _rpc_try(rpc_conn.get_positions)

    # 2) REST fallback
    if not positions:
        r = _rest_get_positions(META_API_TOKEN, ACCOUNT_ID, REGION, symbol)
        if getattr(r, "status_code", 0) == 200:
            try:
                positions = r.json() or []
            except Exception:
                positions = []

    return positions



# --- Close order with RPC, fallback to REST if RPC fails --------------------
async def close_order(df: pd.DataFrame,
                      rpc_conn,
                      symbol: str = SYMBOL,
                      magic: int = 900001,
                      close_col: str = "Close_Trade") -> None:
    if df.empty or close_col not in df.columns:
        return

    sig = df[close_col].iloc[-1]
    if not np.isfinite(sig):
        return

    sides_to_close = {"BUY"} if sig == -1 else {"SELL"}

    try:
        positions = await _pull_positions_all_sources(rpc_conn, symbol)
    except Exception as e:
        print("✘ No se pudieron leer posiciones:", e)
        return
    if not positions:
        return

    def _has_magic(p) -> bool:
        pmagic = p.get("magic", None)
        if pmagic is not None:
            try:
                return int(pmagic) == int(magic)
            except Exception:
                pass
        cmt = str(p.get("comment") or "")
        return f"magic={magic}" in cmt

    def _side_of(p) -> str:
        t = p.get("type")
        if isinstance(t, str):
            tt = t.upper()
            if "BUY" in tt:  return "BUY"
            if "SELL" in tt: return "SELL"
        if t == 0: return "BUY"
        if t == 1: return "SELL"
        return ""

    async def _try_close_rpc(pid: str) -> bool:
        try:
            await asyncio.wait_for(rpc_conn.close_position(pid), timeout=6)
            return True
        except Exception:
            try:
                await asyncio.wait_for(rpc_conn.close_position({"positionId": pid}), timeout=6)
                return True
            except Exception:
                return False

    for p in positions:
        if not _has_magic(p):
            continue
        pid  = str(p.get("id") or p.get("positionId") or "")
        side = _side_of(p)
        if not pid or side not in sides_to_close:
            continue

        ok = await _try_close_rpc(pid)
        if not ok:
            # REST fallback
            r = _rest_close_position(META_API_TOKEN, ACCOUNT_ID, REGION, pid)
            ok = getattr(r, "status_code", 0) == 200

        if ok:
            print(f"✅ Cerrada {side} positionId={pid} (magic={magic})")
        else:
            print(f"✘ No se pudo cerrar {side} positionId={pid} (RPC y REST fallaron)")


async def sync_stop_loss_from_df(df: pd.DataFrame,
                                 rpc_conn,
                                 symbol: str = SYMBOL,
                                 magic: int = 900002,
                                 tol: float = 0.01) -> None:
    """
    Copia desde la posición viva los campos de mercado al DF.
    ¡Importante!: 'profits' solo se escribe en la ÚLTIMA FILA para no
    sobreescribir el histórico de filas previas.
    """
    import re

    if df is None or df.empty:
        return

    _ensure_order_cols(df)

    # 1) Leer posiciones (robusto a SDK sin 'symbol=')
    try:
        positions = await _pull_positions_all_sources(rpc_conn, symbol)
    except Exception:
        positions = []

    if not positions:
        return

    def _has_magic(p) -> bool:
        pm = p.get("magic", None)
        if pm is not None:
            try:
                if int(pm) == int(magic):
                    return True
            except Exception:
                pass
        return f"magic={magic}" in str(p.get("comment") or "")

    def _sym_ok(p) -> bool:
        ps = str(p.get("symbol") or "")
        return (not symbol) or (ps.upper() == str(symbol).upper())

    def _split_comment_magic(cmt: str) -> tuple[str, int | None]:
        if not cmt:
            return "", None
        m = re.search(r"magic\s*=\s*(\d+)", cmt, flags=re.IGNORECASE)
        mag = int(m.group(1)) if m else None
        clean = cmt.split("|", 1)[0].strip()
        return clean, mag

    # Preferir misma symbol + magic; si no, cualquiera con magic
    pos = next((p for p in positions if _has_magic(p) and _sym_ok(p)), None)
    if not pos:
        pos = next((p for p in positions if _has_magic(p)), None)
    if not pos:
        return

    order_id    = str(pos.get("id") or pos.get("positionId") or "")
    magic_val   = pos.get("magic")
    symbol_val  = pos.get("symbol")
    open_price  = pos.get("openPrice") or pos.get("price")
    comment_raw = pos.get("comment") or pos.get("brokerComment") or None
    stop_loss   = pos.get("stopLoss")
    volume_val  = pos.get("volume") or pos.get("lots")
    profit_val  = (pos.get("profit") if pos.get("profit") is not None
                   else pos.get("unrealizedProfit") or pos.get("unrealized_profit"))

    # Type → Long/Short
    t = pos.get("type")
    typ = None
    if isinstance(t, str):
        tu = t.upper()
        if "BUY" in tu:  typ = "Long"
        if "SELL" in tu: typ = "Short"
    elif t == 0:
        typ = "Long"
    elif t == 1:
        typ = "Short"

    entry_dt = pd.to_datetime(pos.get("time"), errors="coerce", utc=True)

    clean_cmt, mag_from_cmt = _split_comment_magic(str(comment_raw or ""))

    # 3) Determinar bloque del trade activo donde escribir (sin tocar 'profits' del bloque)
    block_mask = pd.Series(False, index=df.index)
    if order_id and ("orderId" in df.columns) and df["orderId"].notna().any():
        block_mask = (df["orderId"] == order_id)
    if (not block_mask.any()) and ("Entry_Date" in df.columns) and pd.notna(entry_dt):
        ed = pd.to_datetime(df["Entry_Date"], errors="coerce", utc=True)
        starts = ed.notna() & (ed != ed.shift(1))
        if starts.any():
            last_start = df.index[starts].max()
            block_mask = (df.index >= last_start)
        else:
            block_mask = ed.notna() & (ed >= entry_dt)
    if not block_mask.any():
        block_mask.iloc[-1] = True  # asegura al menos la última fila

    # 4) Escribir valores de mercado en el bloque (EXCEPTO 'profits')
    try:
        if order_id:               df.loc[block_mask, "orderId"]   = str(order_id)
        if magic_val is not None and str(magic_val).strip() != "":
            df.loc[block_mask, "magic"] = int(magic_val)
        elif mag_from_cmt is not None:
            df.loc[block_mask, "magic"] = int(mag_from_cmt)
        else:
            df.loc[block_mask, "magic"] = int(magic)

        if symbol_val:             df.loc[block_mask, "symbol"]    = str(symbol_val)
        if open_price is not None: df.loc[block_mask, "openPrice"] = float(open_price)
        if clean_cmt:              df.loc[block_mask, "comment"]   = clean_cmt
        if typ:                    df.loc[block_mask, "Type"]      = typ
        if pd.notna(entry_dt):     df.loc[block_mask, "Entry_Date"]= entry_dt
        if stop_loss is not None:  df.loc[block_mask, "Real_SL"]   = float(stop_loss)
        if volume_val is not None: df.loc[block_mask, "trade_size"]= float(volume_val)
        # ⛔ NO: df.loc[block_mask, "profits"] = profit_val  (no sobrescribir histórico)
    except Exception:
        pass

    # 5) Escribir SIEMPRE en la ÚLTIMA FILA los valores actuales (incluye 'profits')
    last_idx = df.index[-1]
    if order_id:               df.at[last_idx, "orderId"]   = str(order_id)
    if magic_val is not None and str(magic_val).strip() != "":
        df.at[last_idx, "magic"] = int(magic_val)
    elif mag_from_cmt is not None:
        df.at[last_idx, "magic"] = int(mag_from_cmt)
    else:
        df.at[last_idx, "magic"] = int(magic)

    if symbol_val:             df.at[last_idx, "symbol"]    = str(symbol_val)
    if open_price is not None: df.at[last_idx, "openPrice"] = float(open_price)
    if clean_cmt:              df.at[last_idx, "comment"]   = clean_cmt
    if typ:                    df.at[last_idx, "Type"]      = typ
    if pd.notna(entry_dt):     df.at[last_idx, "Entry_Date"]= entry_dt
    if stop_loss is not None:  df.at[last_idx, "Real_SL"]   = float(stop_loss)
    if volume_val is not None: df.at[last_idx, "trade_size"]= float(volume_val)
    if profit_val is not None: df.at[last_idx, "profits"]   = float(profit_val)


def _last_two_distinct(values: pd.Series) -> tuple[float, float]:
    """
    Devuelve (prev, last) con los dos últimos valores no-NaN distintos.
    Si no hay prev distinto, prev = np.nan.
    """
    s = pd.to_numeric(values, errors="coerce").dropna()
    if s.empty:
        return (np.nan, np.nan)
    last = float(s.iloc[-1])
    prev = float(s[s != last].iloc[-1]) if (s != last).any() else np.nan
    return (prev, last)


# --- FIX 2: find position by magic robustly (with fallbacks) ---------------
async def get_pos_with_magic(rpc_conn, symbol: str, magic: int) -> dict | None:
    positions = await _pull_positions_all_sources(rpc_conn, symbol)
    if not positions:
        return None

    def _has_magic(p) -> bool:
        # magic can be under different keys/types depending on source
        for key in ("magic", "expertMagicNumber", "eaMagicNumber"):
            if key in p and p[key] is not None:
                try:
                    if int(p[key]) == int(magic):
                        return True
                except Exception:
                    # sometimes it's a plain string "900002"
                    if str(p[key]).strip() == str(magic):
                        return True
        # optional safety: encoded in comment (not your case, but harmless)
        if f"magic={magic}" in str(p.get("comment") or ""):
            return True
        return False

    def _sym_ok(p) -> bool:
        ps = str(p.get("symbol") or "")
        return (not symbol) or (ps.upper() == str(symbol).upper())

    # Prefer same symbol first
    for p in positions:
        if _sym_ok(p) and _has_magic(p):
            return p
    # Then any matching magic (in case broker returns different casing/suffix)
    for p in positions:
        if _has_magic(p):
            return p
    return None


# --- FIX 3: stronger fallback in SL sync/modify ----------------------------
async def modify_stoploss_if_changed(df_all: pd.DataFrame,
                                     rpc_conn,
                                     *,
                                     symbol: str,
                                     magic: int,
                                     auth_token: str,
                                     account_id: str,
                                     region: str,
                                     tol: float = 0.0) -> dict:
    prev_sl, last_sl = _last_two_distinct(df_all.get("Stop_Loss_$", pd.Series(dtype=float)))
    if not np.isfinite(last_sl):
        return {"changed": False, "sent": False, "price": np.nan,
                "position_id": "", "status_code": None, "err": "Stop_Loss_$ vacío"}

    changed = (not np.isfinite(prev_sl)) or (abs(last_sl - prev_sl) > tol)
    if not changed:
        return {"changed": False, "sent": False, "price": last_sl,
                "position_id": "", "status_code": None, "err": None}

    # A) try by magic (now robust & with REST fallback)
    pos = await get_pos_with_magic(rpc_conn, symbol=symbol, magic=magic)

    # B) if still not found, try to match the last orderId in the CSV to broker position id
    if not pos:
        last_oid = None
        if "orderId" in df_all.columns and df_all["orderId"].notna().any():
            last_oid = str(df_all["orderId"].dropna().iloc[-1])
        if last_oid:
            positions = await _pull_positions_all_sources(rpc_conn, symbol)
            for p in positions:
                pid = str(p.get("id") or p.get("positionId") or "")
                if pid == last_oid:
                    pos = p
                    break

    if not pos:
        return {"changed": True, "sent": False, "price": last_sl,
                "position_id": "", "status_code": None, "err": "No hay posición con ese magic"}

    position_id = str(pos.get("id") or pos.get("positionId") or "")
    if not position_id:
        return {"changed": True, "sent": False, "price": last_sl,
                "position_id": "", "status_code": None, "err": "positionId vacío"}

    loop = asyncio.get_running_loop()
    resp = await loop.run_in_executor(
        None,
        lambda: _rest_modify_position(
            auth_token=auth_token,
            account_id=account_id,
            region=region,
            position_id=position_id,
            stop_loss=float(last_sl),
            timeout=15
        )
    )
    ok = getattr(resp, "status_code", 0) == 200
    err = None
    if not ok:
        try: err = json.dumps(resp.json())[:300]
        except Exception: err = (getattr(resp, "text", "") or "")[:300]

    return {"changed": True, "sent": ok, "price": last_sl,
            "position_id": position_id, "status_code": getattr(resp, "status_code", None), "err": err}

async def _pull_positions_all_sources(rpc_conn, symbol: str | None):
    positions = []
    # 1) RPC with symbol
    try:
        positions = await rpc_conn.get_positions(symbol=symbol) or []
    except Exception:
        positions = []
    # 2) RPC without symbol
    if not positions:
        try:
            positions = await rpc_conn.get_positions() or []
        except Exception:
            positions = []
    # 3) REST fallback
    if not positions:
        r = _rest_get_positions(META_API_TOKEN, ACCOUNT_ID, REGION, symbol)
        if getattr(r, "status_code", 0) == 200:
            try:
                positions = r.json() or []
            except Exception:
                positions = []
    return positions

# Rutas a listas de columnas (una por lado)
FEATS_LONG_PATH  = f"{root_data}Results/{SYMBOL}_Long_M5M10_ImportantCols.csv"
FEATS_SHORT_PATH = f"{root_data}Results/{SYMBOL}_Short_M5M10_ImportantCols.csv"

def _load_feature_list(path: str) -> list[str]:
    """
    Reads a 1-column CSV (with or without header), removes junk entries like '0',
    'Unnamed:*', '', and de-duplicates while preserving order.
    """
    if not os.path.exists(path):
        print(f"⚠️ Feature list not found: {path}. Falling back to model introspection.")
        return []

    dfc = pd.read_csv(path, header=None)
    # If there is an accidental header or extra columns, take the first column
    if dfc.shape[1] > 1:
        dfc = dfc.iloc[:, [0]]

    raw = dfc.iloc[:, 0].astype(str).tolist()

    def _ok(s: str) -> bool:
        s2 = s.strip()
        if s2 == "" or s2.lower().startswith("unnamed") or s2.lower() == "index" or s2.isdigit():
            return False
        return True

    cleaned = [s.strip() for s in raw if _ok(s)]
    # de-dup preserve order
    seen, out = set(), []
    for c in cleaned:
        if c not in seen:
            out.append(c)
            seen.add(c)
    return out


FEATURES_LONG  = _load_feature_list(FEATS_LONG_PATH)
FEATURES_SHORT = _load_feature_list(FEATS_SHORT_PATH)

def _safe_create_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Uses your create_features(df) on 5m OHLCV (Title-case),
    and ALSO carries through any precomputed 10m feature columns already
    present on `df` (prefix '10min_10min_' or '10_min_10_min').
    """
    try:
        df_in = df.copy()
        col_map = {"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"}
        for lo, hi in col_map.items():
            if lo in df_in.columns and hi not in df_in.columns:
                df_in[hi] = df_in[lo]

        feats = create_features(df_in)  # your big feature builder (5m)

        # carry 10m block that we already appended to df
        extra_cols = [c for c in df.columns
                      if c.startswith("10min_10min_") or c.startswith("10_min_10_min")]
        if extra_cols:
            # ensure same index length; most flows use the df index as the feature index
            feats = feats.reindex(df.index)
            feats = feats.join(df[extra_cols], how="left")

        return feats
    except NameError:
        # if create_features is unavailable, fall back to numeric columns only
        return df.select_dtypes(include=[np.number]).copy()
    except Exception:
        return df.select_dtypes(include=[np.number]).copy()

def _expected_features_from_model(model) -> list[str] | None:
    """
    Best-effort introspection for sklearn-style models to get the exact
    feature name list used at fit time. Returns None if not available.
    """
    # 1) Direct attribute (most sklearn estimators/pipelines after fit)
    for attr in ("feature_names_in_",):
        if hasattr(model, attr):
            try:
                names = list(getattr(model, attr))
                if names:
                    return [str(c) for c in names]
            except Exception:
                pass

    # 2) sklearn Pipeline / Voting / Bagging wrappers
    for attr in ("estimators_", "named_estimators_", "steps", "named_steps"):
        if hasattr(model, attr):
            try:
                obj = getattr(model, attr)
                # pipeline.steps → [('scaler', StandardScaler), ('clf', ...)]
                if isinstance(obj, list):  # steps or list of estimators
                    for _, step in obj:
                        if hasattr(step, "feature_names_in_"):
                            names = list(step.feature_names_in_)
                            if names:
                                return [str(c) for c in names]
                elif isinstance(obj, dict):  # named_steps / named_estimators_
                    for step in obj.values():
                        if hasattr(step, "feature_names_in_"):
                            names = list(step.feature_names_in_)
                            if names:
                                return [str(c) for c in names]
                elif isinstance(obj, (tuple, set)):
                    for step in obj:
                        if hasattr(step, "feature_names_in_"):
                            names = list(step.feature_names_in_)
                            if names:
                                return [str(c) for c in names]
            except Exception:
                pass

    return None


def _align_matrix(X: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """Alinea X a 'cols': agrega faltantes con 0.0 y ordena columnas."""
    if not cols:
        # sin lista → usar todas las columnas disponibles
        Z = X.copy()
        return Z.fillna(0.0)
    missing = [c for c in cols if c not in X.columns]
    if missing:
        for c in missing:
            X[c] = 0.0
        print(f"⚠️ Faltan {len(missing)} features en live: {missing[:8]}{'...' if len(missing)>8 else ''}")
    Z = X.loc[:, cols].copy()
    return Z.fillna(0.0)

def _predict_with_optional_proba(model, X: pd.DataFrame):
    """Devuelve (y_pred, proba, classes) funcionando con sklearn/keras."""
    proba, classes = None, None
    # try predict_proba (sklearn)
    try:
        proba = model.predict_proba(X)
        try:   classes = np.array(model.classes_)
        except Exception: classes = np.arange(proba.shape[1])
    except Exception:
        # fallback keras: predict → proba
        try:
            raw = model.predict(X)
            raw = np.asarray(raw)
            if raw.ndim == 2:
                proba = raw
                if proba.shape[1] == 1:  # binario [p1] → [p0,p1]
                    p1 = proba[:, 0]
                    proba = np.column_stack([1.0 - p1, p1])
                classes = np.arange(proba.shape[1])
        except Exception:
            pass
    # y_pred
    try:
        y_pred = model.predict(X)
        y_pred = np.asarray(y_pred).ravel()
    except Exception:
        if proba is not None:
            y_pred = np.argmax(proba, axis=1)
        else:
            raise RuntimeError("El modelo no soporta predict/predict_proba compatibles.")
    if classes is None and proba is not None:
        classes = np.arange(proba.shape[1])
    return y_pred, proba, classes

# --- REPLACE your run_side_models_inplace with this version ------------
def run_side_models_inplace(
    df: pd.DataFrame,
    *,
    long_model,
    short_model,
    feature_cols_long: list[str] | None,
    feature_cols_short: list[str] | None,
    max_lookback: int = 1200,          # enough for Kalman 1000 + slope/vol windows
    triggers_per_side: int = 1         # only predict at the latest trigger(s)
) -> pd.DataFrame:
    """
    Compute features & run ML ONLY where a trade could open:
      - Open_Trade == +1 → run LONG model at those bars
      - Open_Trade == -1 → run SHORT model at those bars

    We build features on a compact tail slice (max_lookback) and only
    materialize predictions on the trigger rows.
    """

    if df is None or df.empty or ("Open_Trade" not in df.columns):
        return df

    # --- which rows really need predictions?
    idx_long_all  = df.index[df["Open_Trade"] ==  1]
    idx_short_all = df.index[df["Open_Trade"] == -1]

    if len(idx_long_all) == 0 and len(idx_short_all) == 0:
        # ensure prob columns exist, then nothing to do
        _ensure_prob_cols_present(df, "long")
        _ensure_prob_cols_present(df, "short")
        return df

    # take the most recent trigger(s) per side to avoid recomputing everything
    idx_long_targets  = list(idx_long_all[-triggers_per_side:])   if len(idx_long_all)  else []
    idx_short_targets = list(idx_short_all[-triggers_per_side:])  if len(idx_short_all) else []

    # quick exit: if the last trigger rows already have labels, skip
    def _missing_any(pred_col: str, targets: list) -> bool:
        if not targets:
            return False
        if pred_col not in df.columns:
            return True
        return df.loc[targets, pred_col].isna().any()

    need_long  = _missing_any("label_ml_long",  idx_long_targets)
    need_short = _missing_any("label_ml_short", idx_short_targets)
    if not (need_long or need_short):
        _ensure_prob_cols_present(df, "long")
        _ensure_prob_cols_present(df, "short")
        return df

    # ---- pick a compact tail slice to compute features once
    # we’ll include enough history and then only read the rows we care about
    tail = df.tail(max_lookback + 50).copy()

    # 10m block is expensive → compute it only if required by the model’s feature lists
    model_req_long  = _expected_features_from_model(long_model)  or []
    model_req_short = _expected_features_from_model(short_model) or []

    csv_req_long    = _preferred_feature_order("long")  or (feature_cols_long  or [])
    csv_req_short   = _preferred_feature_order("short") or (feature_cols_short or [])

    req_long  = model_req_long  or csv_req_long
    req_short = model_req_short or csv_req_short

    def _needs_10m(required_cols: list[str]) -> bool:
        return any(isinstance(c, str) and (c.startswith("10min_10min_") or c.startswith("10_min_10_min_"))
                   for c in (required_cols or []))

    try:
        if _needs_10m(req_long) or _needs_10m(req_short):
            append_10m_features_inplace(tail, prefix="10min_10min_", pre_scale=True)
    except Exception as e:
        print(f"⚠️ 10m feature block skipped: {e}")

    # Build base feature matrix once on the tail
    feats_all = _safe_create_features(tail)

    # Fix '10min' vs '10_min' aliasing if needed
    feats_all = _ensure_10m_prefix_aliases(feats_all, (req_long or []) + (req_short or []))

    # Ensure Encoded_* columns (if expected)
    enc_n_long  = _encoded_span_from(req_long)
    enc_n_short = _encoded_span_from(req_short)
    for i in range(max(enc_n_long, enc_n_short)):
        c = f"Encoded_{i}"
        if c not in feats_all.columns:
            feats_all[c] = 0.0

    # Align matrices exactly to each side’s requirements
    def _align_exact(X: pd.DataFrame, req: list[str]) -> pd.DataFrame:
        if not req:
            Z = X.select_dtypes(include=[np.number]).copy()
            return Z.fillna(0.0)
        missing = [c for c in req if c not in X.columns]
        for c in missing:
            X[c] = 0.0
        Z = X.loc[:, req].copy()
        return Z.fillna(0.0)

    X_long_all  = _align_exact(feats_all.copy(), req_long)
    X_short_all = _align_exact(feats_all.copy(), req_short)

    # restrict to the trigger rows that actually live inside the tail slice
    tail_idx = feats_all.index
    idx_long  = [i for i in idx_long_targets  if i in tail_idx] if need_long  else []
    idx_short = [i for i in idx_short_targets if i in tail_idx] if need_short else []

    # helper → works with sklearn or keras-like models
    def _predict_with_optional_proba(model, X: pd.DataFrame):
        proba, classes = None, None
        try:
            proba = model.predict_proba(X)
            try:
                classes = np.array(model.classes_)
            except Exception:
                classes = np.arange(proba.shape[1])
        except Exception:
            try:
                raw = model.predict(X)
                raw = np.asarray(raw)
                if raw.ndim == 2:
                    proba = raw
                    if proba.shape[1] == 1:
                        p1 = proba[:, 0]
                        proba = np.column_stack([1.0 - p1, p1])
                    classes = np.arange(proba.shape[1])
            except Exception:
                pass
        try:
            y_pred = model.predict(X)
            y_pred = np.asarray(y_pred).ravel()
        except Exception:
            if proba is not None:
                y_pred = np.argmax(proba, axis=1)
            else:
                raise RuntimeError("El modelo no soporta predict/predict_proba compatibles.")
        if classes is None and proba is not None:
            classes = np.arange(proba.shape[1])
        return y_pred, proba, classes

    # LONG side only where Open_Trade == +1
    if idx_long:
        X = X_long_all.loc[idx_long]
        yL, pL, cL = _predict_with_optional_proba(long_model, X)
        df.loc[idx_long, "label_ml_long"] = yL
        if pL is not None:
            for j in range(pL.shape[1]):
                cname = f"prob_{(cL[j] if cL is not None else j)}_long"
                df.loc[idx_long, cname] = pL[:, j]
        _ensure_prob_cols_present(df, "long")

    # SHORT side only where Open_Trade == -1
    if idx_short:
        X = X_short_all.loc[idx_short]
        yS, pS, cS = _predict_with_optional_proba(short_model, X)
        df.loc[idx_short, "label_ml_short"] = yS
        if pS is not None:
            for j in range(pS.shape[1]):
                cname = f"prob_{(cS[j] if cS is not None else j)}_short"
                df.loc[idx_short, cname] = pS[:, j]
        _ensure_prob_cols_present(df, "short")

    return df


✅ Long model loaded from:  /content/drive/MyDrive/Course Folder/Forex/XAUUSD/Models/BTCUSD_Long_ml_model.joblib
✅ Short model loaded from: /content/drive/MyDrive/Course Folder/Forex/XAUUSD/Models/BTCUSD_Short_ml_model.joblib


In [25]:
# ========================= MAIN (copy/paste) =========================
async def main():
    """
    First run:
      • fetch 900 5m candles
      • compute ATR + signals (+ ML if an entry trigger exists)
      • save ALL 900 rows to CSV (source=1, System_time filled)

    Then:
      • poll at 5m+3s while no position
      • when a position with magic exists, follow every 20s:
          - refresh data, recompute signals, close by Close_Trade
          - recompute atr_close + tick_dyn_atr
          - push SL change if Stop_Loss_$ changed
          - sync market fields from API (incl. Real_SL)
          - save to CSV
    """
    # 0) Connect MetaApi / RPC
    account  = await connect_metaapi(META_API_TOKEN, ACCOUNT_ID)
    rpc_conn = account.get_rpc_connection()
    await rpc_conn.connect()
    asyncio.create_task(collect_10m_data(account))

    # ---- strategy params ----------------------------------------------------
    MAGIC    = 900002
    bars_init = 900  # << first-run bars to compute features on

    length_1 = 300
    length_2 = 410
    length_3 = 710
    length_4 = 870

    smooth_1 = 3
    smooth_2 = 3
    smooth_3 = 3
    smooth_4 = 5

    # ---- small helpers (local) ---------------------------------------------
    def _frame_delta(tf: str) -> dt.timedelta:
        tf = (tf or "").strip().lower()
        if tf.endswith("mn"):
            tf = tf[:-2] + "m"
        if tf.endswith("m"):
            return dt.timedelta(minutes=int(tf[:-1] or "1"))
        if tf.endswith("h"):
            return dt.timedelta(hours=int(tf[:-1] or "1"))
        if tf.endswith("d"):
            return dt.timedelta(days=int(tf[:-1] or "1"))
        return dt.timedelta(minutes=1)

    def _floor_to_frame(ts: dt.datetime, delta: dt.timedelta) -> dt.datetime:
        if ts.tzinfo is None:
            ts = ts.replace(tzinfo=dt.timezone.utc)
        epoch = dt.datetime(1970, 1, 1, tzinfo=dt.timezone.utc)
        secs  = int((ts - epoch).total_seconds())
        step  = int(delta.total_seconds()) or 60
        return epoch + dt.timedelta(seconds=(secs // step) * step)

    async def _has_open_position_magic(rpc_conn, symbol: str, magic: int) -> bool:
        # try RPC
        try:
            positions = await rpc_conn.get_positions(symbol=symbol) or []
        except Exception:
            positions = []
        # REST fallback
        if not positions:
            r = _rest_get_positions(META_API_TOKEN, ACCOUNT_ID, REGION, symbol)
            if getattr(r, "status_code", 0) == 200:
                try:
                    positions = r.json() or []
                except Exception:
                    positions = []
        if not positions:
            return False

        for p in positions:
            pm = p.get("magic")
            has_magic = False
            if pm is not None:
                try:
                    has_magic = int(pm) == int(magic)
                except Exception:
                    has_magic = False
            if not has_magic and f"magic={magic}" in str(p.get("comment") or ""):
                has_magic = True
            if has_magic:
                return True
        return False

    async def _get_api_type(rpc_conn, symbol: str, magic: int) -> Optional[str]:
        try:
            positions = await rpc_conn.get_positions(symbol=symbol) or []
        except Exception:
            positions = []
        def _has_magic(p):
            pm = p.get("magic", None)
            ok = False
            if pm is not None:
                try:
                    ok = int(pm) == int(magic)
                except Exception:
                    ok = False
            if not ok:
                ok = f"magic={magic}" in str(p.get("comment") or "")
            return ok
        def _side_of(p):
            t = p.get("type")
            if isinstance(t, str):
                tt = t.upper()
                if "BUY" in tt:  return "BUY"
                if "SELL" in tt: return "SELL"
            if t == 0: return "BUY"
            if t == 1: return "SELL"
            return ""
        for p in positions:
            if not _has_magic(p):
                continue
            s = _side_of(p)
            if s:
                return "Long" if s == "BUY" else "Short"
        return None

    def _sync_type_in_df(df_all: pd.DataFrame, api_type: Optional[str]) -> None:
        if not api_type or df_all.empty:
            return
        last_oid = df_all.get("orderId")
        if last_oid is not None and last_oid.notna().any():
            last_oid_val = last_oid.dropna().iloc[-1]
            mask = df_all["orderId"] == last_oid_val
        else:
            ed = pd.to_datetime(df_all.get("Entry_Date"), errors="coerce", utc=True)
            starts = ed.notna() & (ed != ed.shift(1))
            if starts.any():
                start_idx = df_all.index[starts][-1]
                mask = (df_all.index >= start_idx)
            else:
                mask = pd.Series(False, index=df_all.index)
        if mask.any():
            df_all.loc[mask, "Type"] = api_type
        else:
            df_all.at[df_all.index[-1], "Type"] = api_type

    # 1) First-run: create/migrate CSV ----------------------------------------
    if not os.path.exists(FILE_PATH):
        # --- fetch 900 bars, compute, and SAVE ALL ROWS ---
        df_hist = await get_candles_5m(account, start=None, limit=bars_init)
        df_hist = (df_hist
                   .drop_duplicates("time")
                   .sort_values("time")
                   .reset_index(drop=True))

        if len(df_hist) >= 14:
            df_hist["ATR"] = ta.ATR(df_hist["high"], df_hist["low"], df_hist["close"], 14).round(4)

        df_all = df_hist.copy()

        generate_trade_signals(
            df_all, length_1, length_2, length_3, length_4,
            smooth_1, smooth_2, smooth_3, smooth_4
        )

        maybe_compute_ml_for_entry(
            df_all,
            long_model=long_ml_model,
            short_model=short_ml_model,
            feature_cols_long=FEATURES_LONG,
            feature_cols_short=FEATURES_SHORT
        )
        sync_unified_ml_cols(df_all)
        _ensure_order_cols(df_all)

        # mark provenance and stamp time on every row
        df_all["source"] = 1
        stamp_system_time(df_all, mode="missing")

        save_csv(df_all)
        print(f"✔ Archivo inicial creado: {len(df_all)} filas guardadas (se analizaron {bars_init} velas)")
    else:
        migrate_csv_if_needed(FILE_PATH)

    # 2) Live loop -------------------------------------------------------------
    while True:

        # ───────── NO open position → wait to next 5m+03s tick ─────────
        if not await _has_open_position_magic(rpc_conn, SYMBOL, MAGIC):
            await asyncio.sleep(FETCH_INTERVAL)

            now_utc   = dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc)
            delta     = _frame_delta(time_frame_data)
            this_bar  = _floor_to_frame(now_utc, delta)
            prev_bar  = this_bar - delta

            # fetch the last closed bar only
            df_new = await get_candles_5m(account, start=None, limit=50)
            df_new = (df_new[df_new["time"] <= prev_bar]
                      .drop_duplicates("time")
                      .sort_values("time"))
            if not df_new.empty:
                df_new = df_new.iloc[[-1]]

            df_all = _load_csv()
            existing_times = set(pd.to_datetime(df_all["time"], utc=True)) if not df_all.empty and "time" in df_all.columns else set()
            if df_all.empty:
                df_all = df_new.copy()
            else:
                df_all = (pd.concat([df_all, df_new], ignore_index=True)
                            .drop_duplicates("time")
                            .sort_values("time")
                            .reset_index(drop=True))

            if len(df_all) >= 14:
                df_all["ATR"] = ta.ATR(df_all["high"], df_all["low"], df_all["close"], 14).round(4)

            generate_trade_signals(
                df_all, length_1, length_2, length_3, length_4,
                smooth_1, smooth_2, smooth_3, smooth_4
            )
            maybe_compute_ml_for_entry(
                df_all,
                long_model=long_ml_model,
                short_model=short_ml_model,
                feature_cols_long=FEATURES_LONG,
                feature_cols_short=FEATURES_SHORT
            )
            _ensure_order_cols(df_all)
            sync_unified_ml_cols(df_all)

            # mark new row(s) as source=1
            if not df_new.empty:
                new_times = set(pd.to_datetime(df_new["time"], utc=True)) - existing_times
                if new_times:
                    df_all.loc[pd.to_datetime(df_all["time"], utc=True).isin(new_times), "source"] = 1

            # try to open trade (if ML permits)
            await open_trade(df_all, rpc_conn, symbol=SYMBOL, lot=LOT, comment=COMMENT, magic=MAGIC)

            # sync type and market fields, then save
            api_type = await _get_api_type(rpc_conn, SYMBOL, MAGIC)
            _sync_type_in_df(df_all, api_type)
            await sync_stop_loss_from_df(df_all, rpc_conn, symbol=SYMBOL, magic=MAGIC)

            stamp_system_time(df_all, "last")
            save_csv(df_all)
            print(dt.datetime.utcnow().strftime("%H:%M:%S"),
                  "| actualización (modo búsqueda de entrada)")

        # ───────── OPEN position → follow every 20s ─────────
        else:
            print("▶ Modo seguimiento: posición con magic 900002 detectada")

            while await _has_open_position_magic(rpc_conn, SYMBOL, MAGIC):

                await asyncio.sleep(FETCH_INTERVAL)

                # load CSV and last time
                df_all = _load_csv()
                last_time = None
                if not df_all.empty and "time" in df_all.columns:
                    last_time = pd.to_datetime(df_all["time"], errors="coerce", utc=True).max()

                # live snapshot; append only strictly new timestamps
                df_new = await get_current_candle_snapshot(account, rpc_conn, symbol=SYMBOL, timeframe=time_frame_data)
                df_new = df_new.drop_duplicates("time").sort_values("time")
                df_inc = df_new[df_new["time"] > last_time] if last_time is not None and pd.notna(last_time) else df_new

                if not df_inc.empty:
                    df_all = pd.concat([df_all, df_inc], ignore_index=True)

                # recalc ATR and signals
                if len(df_all) >= 14:
                    df_all["ATR"] = ta.ATR(df_all["high"], df_all["low"], df_all["close"], 14).round(4)

                generate_trade_signals(
                    df_all, length_1, length_2, length_3, length_4,
                    smooth_1, smooth_2, smooth_3, smooth_4
                )
                maybe_compute_ml_for_entry(
                    df_all,
                    long_model=long_ml_model,
                    short_model=short_ml_model,
                    feature_cols_long=FEATURES_LONG,
                    feature_cols_short=FEATURES_SHORT
                )
                sync_unified_ml_cols(df_all)

                # close order if Close_Trade says so
                await close_order(df_all, rpc_conn, symbol=SYMBOL, magic=MAGIC, close_col="Close_Trade")

                # recompute ATR-based tracking + dynamic SL
                atr_close(df_all)
                tick_dyn_atr(df_all)

                # if Stop_Loss_$ changed, push modification
                sl_res = await modify_stoploss_if_changed(
                    df_all, rpc_conn,
                    symbol=SYMBOL, magic=MAGIC,
                    auth_token=META_API_TOKEN,
                    account_id=ACCOUNT_ID,
                    region=REGION,
                    tol=0.0
                )

                # always sync market fields (including Real_SL) from API
                await sync_stop_loss_from_df(df_all, rpc_conn, symbol=SYMBOL, magic=MAGIC)

                # stamp and mark source for new rows, then save
                stamp_system_time(df_all, "last")
                _ensure_order_cols(df_all)
                if not df_inc.empty:
                    df_all.loc[df_all["time"].isin(df_inc["time"]), "source"] = 9
                else:
                    df_all.at[df_all.index[-1], "source"] = 9
                save_csv(df_all)

                # logs
                base = (dt.datetime.utcnow().strftime("%H:%M:%S")
                        + " | seguimiento 20s: señales, posible cierre, atr_close + tick_dyn_atr + sync_market")
                print(base)
                if sl_res.get("changed"):
                    if sl_res.get("sent"):
                        print(f"   ↳ SL actualizado en broker a {sl_res['price']:.2f} (positionId={sl_res.get('position_id','?')})")
                    else:
                        msg_err = sl_res.get("err") or "error desconocido"
                        price = sl_res.get("price", np.nan)
                        print(f"   ↳ SL cambió a {price:.2f} pero no se envió ({msg_err})")

            print("⏹ La posición con magic 900002 se cerró → regreso a búsqueda de entrada")

In [27]:
# Background task to store 10-minute candles
import os
import pandas as pd
import datetime as dt
import asyncio

FILE_PATH_10M = "xauusd_data_10m.csv"
BARS_INIT_10M = 900

async def collect_10m_data(account, symbol=SYMBOL, path=FILE_PATH_10M):
    """Fetch initial 10m candles (900 bars) and append new candle every 10 minutes."""
    if not os.path.exists(path):
        candles = await account.get_historical_candles(symbol=symbol, timeframe='10m', start_time=None, limit=BARS_INIT_10M)
        df = pd.DataFrame([{ 'time': pd.to_datetime(c['time'], utc=True), 'open': c['open'], 'high': c['high'], 'low': c['low'], 'close': c['close'], 'volume': c.get('volume'), 'tickVolume': c.get('tickVolume'), 'spread': c.get('spread') } for c in candles])
        if not df.empty:
            df['time'] = pd.to_datetime(df['time'], utc=True).dt.strftime('%Y-%m-%d %H:%M:%S')
            df.to_csv(path, index=False)
    while True:
        await asyncio.sleep(600)
        candles = await account.get_historical_candles(symbol=symbol, timeframe='10m', start_time=None, limit=1)
        if candles:
            c = candles[-1]
            row = pd.DataFrame([{ 'time': pd.to_datetime(c['time'], utc=True), 'open': c['open'], 'high': c['high'], 'low': c['low'], 'close': c['close'], 'volume': c.get('volume'), 'tickVolume': c.get('tickVolume'), 'spread': c.get('spread') }])
            row['time'] = pd.to_datetime(row['time'], utc=True).dt.strftime('%Y-%m-%d %H:%M:%S')
            if os.path.exists(path):
                existing = pd.read_csv(path)
                existing_times = pd.to_datetime(existing['time'], errors='coerce')
                row_time = pd.to_datetime(row['time']).iloc[0]
                if row_time not in existing_times:
                    row.to_csv(path, mode='a', header=False, index=False)
            else:
                row.to_csv(path, index=False)


# Run

In [28]:
###############################################################################
# EJECUCIÓN

if __name__ == "__main__":
    nest_asyncio.apply()     # solo en notebooks
    asyncio.run(main())

ERROR:SubscriptionManager:163d9a57-1f07-4e78-a6af-036efe867c1b:0: Failed to subscribe {"name": "TimeoutException", "message": "It seems like the account 163d9a57-1f07-4e78-a6af-036efe867c1b is not connected to broker yet or SDK settings you use does not match the account region. Please make sure account is connected to broker before retrying the request. Please make sure you pass a region option to MetaApi constructor for Java SDK. Please make sure you do not pass region option to MetaApi constructor for javascript and python SDKs.", "trace": "Traceback (most recent call last):\n  File \"/usr/local/lib/python3.12/dist-packages/metaapi_cloud_sdk/clients/metaapi/subscription_manager.py\", line 110, in subscribe_task\n    await self.subscribe(account_id, instance_number)\n  File \"/usr/local/lib/python3.12/dist-packages/metaapi_cloud_sdk/clients/metaapi/metaapi_websocket_client.py\", line 1625, in rpc_request\n    return await self._make_request(account_id, instance_number, request, timeo

[2025-09-09T21:53:12.875653] 163d9a57-1f07-4e78-a6af-036efe867c1b:0: Failed to subscribe {"name": "TimeoutException", "message": "It seems like the account 163d9a57-1f07-4e78-a6af-036efe867c1b is not connected to broker yet or SDK settings you use does not match the account region. Please make sure account is connected to broker before retrying the request. Please make sure you pass a region option to MetaApi constructor for Java SDK. Please make sure you do not pass region option to MetaApi constructor for javascript and python SDKs.", "trace": "Traceback (most recent call last):\n  File \"/usr/local/lib/python3.12/dist-packages/metaapi_cloud_sdk/clients/metaapi/subscription_manager.py\", line 110, in subscribe_task\n    await self.subscribe(account_id, instance_number)\n  File \"/usr/local/lib/python3.12/dist-packages/metaapi_cloud_sdk/clients/metaapi/metaapi_websocket_client.py\", line 1625, in rpc_request\n    return await self._make_request(account_id, instance_number, request, ti

ERROR:SubscriptionManager:163d9a57-1f07-4e78-a6af-036efe867c1b:0: Failed to subscribe {"name": "TimeoutException", "message": "It seems like the account 163d9a57-1f07-4e78-a6af-036efe867c1b is not connected to broker yet or SDK settings you use does not match the account region. Please make sure account is connected to broker before retrying the request. Please make sure you pass a region option to MetaApi constructor for Java SDK. Please make sure you do not pass region option to MetaApi constructor for javascript and python SDKs.", "trace": "Traceback (most recent call last):\n  File \"/usr/local/lib/python3.12/dist-packages/metaapi_cloud_sdk/clients/metaapi/subscription_manager.py\", line 110, in subscribe_task\n    await self.subscribe(account_id, instance_number)\n  File \"/usr/local/lib/python3.12/dist-packages/metaapi_cloud_sdk/clients/metaapi/metaapi_websocket_client.py\", line 1625, in rpc_request\n    return await self._make_request(account_id, instance_number, request, timeo

KeyboardInterrupt: 